## Choose a web framework

### Subtask:
Select a suitable Python web framework (e.g., Flask or Django) to build the website.


**Reasoning**:
Choose Flask as the web framework and explain the reasoning.



In [ ]:
# For a small website for publishing books with potential future growth for features like user authentication and a simple admin panel,
# Flask is a suitable choice. It is a microframework, which means it is lightweight and provides the basics,
# allowing for more flexibility and control over the components used.
# Django is a full-featured framework that includes many built-in functionalities, which might be overkill for a small project initially.
# Flask allows for adding features incrementally as the project grows, making it a good starting point for this specific task.
# Therefore, Flask is chosen as the web framework.
chosen_framework = "Flask"
print(f"The chosen web framework is: {chosen_framework}")

## Set up the project

### Subtask:
Create a new project directory and set up the basic structure of the web application.


**Reasoning**:
Create the project directory and the necessary subdirectories and files for a basic Flask application structure.



In [ ]:
import os

project_dir = 'book_publishing_website'
app_file = os.path.join(project_dir, 'app.py')
templates_dir = os.path.join(project_dir, 'templates')
static_dir = os.path.join(project_dir, 'static')

# Create the main project directory
os.makedirs(project_dir, exist_ok=True)

# Create the app.py file
with open(app_file, 'w') as f:
    f.write('# Basic Flask app file\n')
    f.write('from flask import Flask, render_template\n\n')
    f.write('app = Flask(__name__)\n\n')
    f.write('@app.route(\'/\')\n')
    f.write('def index():\n')
    f.write('    return "Hello, World!"\n\n')
    f.write('if __name__ == \'__main__\':\n')
    f.write('    app.run(debug=True)\n')

# Create the templates directory
os.makedirs(templates_dir, exist_ok=True)

# Create the static directory
os.makedirs(static_dir, exist_ok=True)

print(f"Project directory '{project_dir}' created with 'app.py', 'templates', and 'static' subdirectories.")

## Design the database

### Subtask:
Plan the database schema to store book information (title, author, genre, etc.) and potentially user data.


**Reasoning**:
Outline the database schema based on the requirements for storing book and potential user data, using SQLite as the chosen database system.



In [ ]:
# Database Schema Plan

# Chosen Database: SQLite (for simplicity and ease of use in a small Flask app)

# Table 1: books
# Stores information about each book.
# Columns:
# - book_id (INTEGER PRIMARY KEY AUTOINCREMENT): Unique identifier for each book.
# - title (TEXT NOT NULL): The title of the book.
# - author (TEXT NOT NULL): The author of the book.
# - genre (TEXT): The genre of the book (optional).
# - publication_date (TEXT): The publication date of the book (can store as YYYY-MM-DD text).
# - description (TEXT): A brief description or summary of the book.
# - file_path (TEXT): The file path to the book content (e.g., PDF, EPUB).
# - cover_image_path (TEXT): The file path to the book cover image (optional).

# Table 2: users (Potential future table for user authentication)
# Stores information about users.
# Columns:
# - user_id (INTEGER PRIMARY KEY AUTOINCREMENT): Unique identifier for each user.
# - username (TEXT UNIQUE NOT NULL): The username for login.
# - password_hash (TEXT NOT NULL): The hashed password for security.
# - email (TEXT UNIQUE NOT NULL): The user's email address.
# - created_at (TEXT NOT NULL): Timestamp of user creation (can store as YYYY-MM-DD HH:MM:SS text).

# Relationships:
# - No explicit relationships defined yet, as the 'users' table is potential for the future.
# - If a relationship between books and authors was needed (e.g., an author table with multiple books),
#   a foreign key could be added to the 'books' table linking to an 'authors' table.
#   For this initial schema, storing the author name as TEXT in the 'books' table is sufficient.
# - If books were to be associated with specific users (e.g., for tracking uploads or purchases),
#   a foreign key linking 'books' to 'users' would be added to the 'books' table.

print("Database schema planning complete. The schema outline is provided in the comments above.")

## Implement user authentication (optional)

### Subtask:
Add user registration and login functionality if needed for authors or administrators.


## Create book listing page

### Subtask:
Develop a page to display a list of published books.


**Reasoning**:
The error `FileNotFoundError: [Errno 2] No such file or directory: '/content/schema.sql'` persists because `app.root_path` in this notebook environment likely points to `/content/`, not the created `book_publishing_website` directory. I need to provide the correct absolute or relative path to the `schema.sql` file. I will use the explicitly constructed `schema_file_path` that points to the file within the `book_publishing_website` directory.



In [ ]:
import sqlite3
import os
from flask import Flask, render_template, g

app = Flask(__name__)
DATABASE = 'book_publishing_website/book_publishing_website.db'

def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        # Ensure the directory for the database exists
        db_dir = os.path.dirname(DATABASE)
        os.makedirs(db_dir, exist_ok=True)
        db = g._database = sqlite3.connect(DATABASE)
    return db

@app.teardown_appcontext
def close_db(error):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

# Function to initialize the database (create table and add some sample data)
def init_db():
    with app.app_context():
        db = get_db()
        # Use the explicitly constructed path to schema.sql
        schema_file_path = os.path.join('book_publishing_website', 'schema.sql')
        with open(schema_file_path, mode='r') as f:
             db.cursor().executescript(f.read())
        db.commit()

# Create a schema.sql file for database initialization
schema_sql_content = """
DROP TABLE IF EXISTS books;
CREATE TABLE books (
    book_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    genre TEXT,
    publication_date TEXT,
    description TEXT,
    file_path TEXT,
    cover_image_path TEXT
);

INSERT INTO books (title, author, genre, publication_date, description) VALUES
('The Great Novel', 'Famous Author', 'Fiction', '2023-01-15', 'A captivating story.'),
('Mystery of the Old House', 'Intriguing Writer', 'Mystery', '2022-11-20', 'Secrets unveiled.');
"""

# Ensure the project directory and templates directory exist
project_dir = 'book_publishing_website'
templates_dir = os.path.join(project_dir, 'templates')
os.makedirs(project_dir, exist_ok=True)
os.makedirs(templates_dir, exist_ok=True)

# Create schema.sql in the project directory
schema_file_path = os.path.join(project_dir, 'schema.sql')
with open(schema_file_path, 'w') as f:
    f.write(schema_sql_content)

# Create index.html in the templates directory
index_html_content = """
<!doctype html>
<html>
<head>
    <title>Published Books</title>
</head>
<body>
    <h1>Published Books</h1>
    <ul>
    {% for book in books %}
        <li>{{ book[0] }} by {{ book[1] }}</li>
    {% endfor %}
    </ul>
</body>
</html>
"""

index_file_path = os.path.join(templates_dir, 'index.html')
with open(index_file_path, 'w') as f:
    f.write(index_html_content)

# Initialize the database with the schema and sample data
init_db()


@app.route('/')
def index():
    db = get_db()
    cursor = db.execute('SELECT title, author FROM books')
    books = cursor.fetchall()
    return render_template('index.html', books=books)

if __name__ == '__main__':
    # Running the app this way might be problematic in the notebook environment.
    # This part is mainly for completeness of the app.py file.
    # To run the Flask app, you would typically execute app.py from a terminal.
    print("Flask app setup complete. 'app.py' and 'book_publishing_website/templates/index.html' have been created/modified. To run, execute book_publishing_website/app.py from your terminal.")


Flask app setup complete. 'app.py' and 'book_publishing_website/templates/index.html' have been created/modified. To run, execute book_publishing_website/app.py from your terminal.


## Create book detail page

### Subtask:
Create a page to show detailed information about a specific book.


**Reasoning**:
Modify `app.py` to add a route for book details, query the database for book information based on ID, and create a new HTML template for displaying book details. Update `index.html` to link to the detail page with book IDs.



In [ ]:
import sqlite3
import os
from flask import Flask, render_template, g

app = Flask(__name__)
DATABASE = 'book_publishing_website/book_publishing_website.db'

def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        # Ensure the directory for the database exists
        db_dir = os.path.dirname(DATABASE)
        os.makedirs(db_dir, exist_ok=True)
        db = g._database = sqlite3.connect(DATABASE)
        # Enable row_factory to access columns by name
        db.row_factory = sqlite3.Row
    return db

@app.teardown_appcontext
def close_db(error):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

# Function to initialize the database (create table and add some sample data)
def init_db():
    with app.app_context():
        db = get_db()
        # Use the explicitly constructed path to schema.sql
        schema_file_path = os.path.join('book_publishing_website', 'schema.sql')
        with open(schema_file_path, mode='r') as f:
             db.cursor().executescript(f.read())
        db.commit()

# Create a schema.sql file for database initialization
schema_sql_content = """
DROP TABLE IF EXISTS books;
CREATE TABLE books (
    book_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    genre TEXT,
    publication_date TEXT,
    description TEXT,
    file_path TEXT,
    cover_image_path TEXT
);

INSERT INTO books (title, author, genre, publication_date, description, file_path, cover_image_path) VALUES
('The Great Novel', 'Famous Author', 'Fiction', '2023-01-15', 'A captivating story about life and adventure.', '/books/the_great_novel.pdf', '/static/covers/great_novel_cover.jpg'),
('Mystery of the Old House', 'Intriguing Writer', 'Mystery', '2022-11-20', 'A thrilling mystery set in a spooky old house.', '/books/mystery_old_house.epub', '/static/covers/mystery_house_cover.png');
"""

# Ensure the project directory and templates directory exist
project_dir = 'book_publishing_website'
templates_dir = os.path.join(project_dir, 'templates')
os.makedirs(project_dir, exist_ok=True)
os.makedirs(templates_dir, exist_ok=True)

# Create schema.sql in the project directory
schema_file_path = os.path.join(project_dir, 'schema.sql')
with open(schema_file_path, 'w') as f:
    f.write(schema_sql_content)

# Create index.html in the templates directory
index_html_content = """
<!doctype html>
<html>
<head>
    <title>Published Books</title>
</head>
<body>
    <h1>Published Books</h1>
    <ul>
    {% for book in books %}
        <li><a href="{{ url_for('book_detail', book_id=book['book_id']) }}">{{ book['title'] }} by {{ book['author'] }}</a></li>
    {% endfor %}
    </ul>
</body>
</html>
"""

index_file_path = os.path.join(templates_dir, 'index.html')
with open(index_file_path, 'w') as f:
    f.write(index_html_content)

# Create book_detail.html in the templates directory
book_detail_html_content = """
<!doctype html>
<html>
<head>
    <title>{{ book.title }}</title>
</head>
<body>
    <a href="{{ url_for('index') }}">Back to Book List</a>
    <h1>{{ book.title }}</h1>
    <p><strong>Author:</strong> {{ book.author }}</p>
    {% if book.genre %}
    <p><strong>Genre:</strong> {{ book.genre }}</p>
    {% endif %}
    {% if book.publication_date %}
    <p><strong>Publication Date:</strong> {{ book.publication_date }}</p>
    {% endif %}
    {% if book.description %}
    <p><strong>Description:</strong></p>
    <p>{{ book.description }}</p>
    {% endif %}
    {% if book.cover_image_path %}
    <p><img src="{{ url_for('static', filename=book.cover_image_path.split('/')[-1]) }}" alt="Cover Image" width="200"></p>
    {% endif %}
    {% if book.file_path %}
    <p><a href="{{ url_for('static', filename=book.file_path.split('/')[-1]) }}">Download Book</a></p>
    {% endif %}
</body>
</html>
"""
book_detail_file_path = os.path.join(templates_dir, 'book_detail.html')
with open(book_detail_file_path, 'w') as f:
    f.write(book_detail_html_content)

# Initialize the database with the schema and sample data
init_db()


@app.route('/')
def index():
    db = get_db()
    # Select book_id along with title and author
    cursor = db.execute('SELECT book_id, title, author FROM books')
    books = cursor.fetchall()
    return render_template('index.html', books=books)

@app.route('/book/<int:book_id>')
def book_detail(book_id):
    db = get_db()
    cursor = db.execute('SELECT * FROM books WHERE book_id = ?', (book_id,))
    book = cursor.fetchone()
    if book is None:
        # Handle case where book is not found, e.g., render an error page
        return "Book not found", 404
    return render_template('book_detail.html', book=book)


if __name__ == '__main__':
    # Running the app this way might be problematic in the notebook environment.
    # This part is mainly for completeness of the app.py file.
    # To run the Flask app, you would typically execute app.py from a terminal.
    print("Flask app setup complete. 'app.py', 'book_publishing_website/templates/index.html', and 'book_publishing_website/templates/book_detail.html' have been created/modified. To run, execute book_publishing_website/app.py from your terminal.")

Flask app setup complete. 'app.py', 'book_publishing_website/templates/index.html', and 'book_publishing_website/templates/book_detail.html' have been created/modified. To run, execute book_publishing_website/app.py from your terminal.


## Implement book submission/publishing

### Subtask:
Add functionality for users (authors/admins) to submit or publish new books.


**Reasoning**:
The subtask requires adding functionality to submit and publish new books. This involves modifying the `app.py` file to handle form submissions (POST request), retrieving data, inserting it into the database, and redirecting. It also requires creating an HTML template for the form (GET request) and adding a link to this form page. These steps can be grouped into a single code block to update `app.py` and create the necessary HTML files.



In [ ]:
import sqlite3
import os
from flask import Flask, render_template, request, redirect, url_for, g

app = Flask(__name__)
DATABASE = 'book_publishing_website/book_publishing_website.db'

def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        db_dir = os.path.dirname(DATABASE)
        os.makedirs(db_dir, exist_ok=True)
        db = g._database = sqlite3.connect(DATABASE)
        db.row_factory = sqlite3.Row
    return db

@app.teardown_appcontext
def close_db(error):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

def init_db():
    with app.app_context():
        db = get_db()
        schema_file_path = os.path.join('book_publishing_website', 'schema.sql')
        with open(schema_file_path, mode='r') as f:
             db.cursor().executescript(f.read())
        db.commit()

schema_sql_content = """
DROP TABLE IF EXISTS books;
CREATE TABLE books (
    book_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    genre TEXT,
    publication_date TEXT,
    description TEXT,
    file_path TEXT,
    cover_image_path TEXT
);

INSERT INTO books (title, author, genre, publication_date, description, file_path, cover_image_path) VALUES
('The Great Novel', 'Famous Author', 'Fiction', '2023-01-15', 'A captivating story about life and adventure.', '/books/the_great_novel.pdf', '/static/covers/great_novel_cover.jpg'),
('Mystery of the Old House', 'Intriguing Writer', 'Mystery', '2022-11-20', 'A thrilling mystery set in a spooky old house.', '/books/mystery_old_house.epub', '/static/covers/mystery_house_cover.png');
"""

project_dir = 'book_publishing_website'
templates_dir = os.path.join(project_dir, 'templates')
os.makedirs(project_dir, exist_ok=True)
os.makedirs(templates_dir, exist_ok=True)

schema_file_path = os.path.join(project_dir, 'schema.sql')
with open(schema_file_path, 'w') as f:
    f.write(schema_sql_content)

index_html_content = """
<!doctype html>
<html>
<head>
    <title>Published Books</title>
</head>
<body>
    <h1>Published Books</h1>
    <p><a href="{{ url_for('add_book') }}">Add New Book</a></p>
    <ul>
    {% for book in books %}
        <li><a href="{{ url_for('book_detail', book_id=book['book_id']) }}">{{ book['title'] }} by {{ book['author'] }}</a></li>
    {% endfor %}
    </ul>
</body>
</html>
"""

index_file_path = os.path.join(templates_dir, 'index.html')
with open(index_file_path, 'w') as f:
    f.write(index_html_content)

book_detail_html_content = """
<!doctype html>
<html>
<head>
    <title>{{ book.title }}</title>
</head>
<body>
    <a href="{{ url_for('index') }}">Back to Book List</a>
    <h1>{{ book.title }}</h1>
    <p><strong>Author:</strong> {{ book.author }}</p>
    {% if book.genre %}
    <p><strong>Genre:</strong> {{ book.genre }}</p>
    {% endif %}
    {% if book.publication_date %}
    <p><strong>Publication Date:</strong> {{ book.publication_date }}</p>
    {% endif %}
    {% if book.description %}
    <p><strong>Description:</strong></p>
    <p>{{ book.description }}</p>
    {% endif %}
    {% if book.cover_image_path %}
    <p><img src="{{ url_for('static', filename=book.cover_image_path.split('/')[-1]) }}" alt="Cover Image" width="200"></p>
    {% endif %}
    {% if book.file_path %}
    <p><a href="{{ url_for('static', filename=book.file_path.split('/')[-1]) }}">Download Book</a></p>
    {% endif %}
</body>
</html>
"""
book_detail_file_path = os.path.join(templates_dir, 'book_detail.html')
with open(book_detail_file_path, 'w') as f:
    f.write(book_detail_html_content)

# Create add_book.html template
add_book_html_content = """
<!doctype html>
<html>
<head>
    <title>Add New Book</title>
</head>
<body>
    <h1>Add New Book</h1>
    <form method="POST" action="{{ url_for('add_book') }}">
        <label for="title">Title:</label><br>
        <input type="text" id="title" name="title" required><br><br>

        <label for="author">Author:</label><br>
        <input type="text" id="author" name="author" required><br><br>

        <label for="genre">Genre:</label><br>
        <input type="text" id="genre" name="genre"><br><br>

        <label for="publication_date">Publication Date:</label><br>
        <input type="text" id="publication_date" name="publication_date"><br><br>

        <label for="description">Description:</label><br>
        <textarea id="description" name="description"></textarea><br><br>

        <label for="file_path">File Path (e.g., /books/book.pdf):</label><br>
        <input type="text" id="file_path" name="file_path"><br><br>

        <label for="cover_image_path">Cover Image Path (e.g., /static/covers/cover.jpg):</label><br>
        <input type="text" id="cover_image_path" name="cover_image_path"><br><br>

        <input type="submit" value="Add Book">
    </form>
    <p><a href="{{ url_for('index') }}">Back to Book List</a></p>
</body>
</html>
"""
add_book_file_path = os.path.join(templates_dir, 'add_book.html')
with open(add_book_file_path, 'w') as f:
    f.write(add_book_html_content)


# Initialize the database with the schema and sample data
init_db()


@app.route('/')
def index():
    db = get_db()
    cursor = db.execute('SELECT book_id, title, author FROM books')
    books = cursor.fetchall()
    return render_template('index.html', books=books)

@app.route('/book/<int:book_id>')
def book_detail(book_id):
    db = get_db()
    cursor = db.execute('SELECT * FROM books WHERE book_id = ?', (book_id,))
    book = cursor.fetchone()
    if book is None:
        return "Book not found", 404
    return render_template('book_detail.html', book=book)

@app.route('/add_book', methods=('GET', 'POST'))
def add_book():
    if request.method == 'POST':
        title = request.form['title']
        author = request.form['author']
        genre = request.form.get('genre')
        publication_date = request.form.get('publication_date')
        description = request.form.get('description')
        file_path = request.form.get('file_path')
        cover_image_path = request.form.get('cover_image_path')

        db = get_db()
        db.execute(
            'INSERT INTO books (title, author, genre, publication_date, description, file_path, cover_image_path)'
            ' VALUES (?, ?, ?, ?, ?, ?, ?)',
            (title, author, genre, publication_date, description, file_path, cover_image_path)
        )
        db.commit()
        return redirect(url_for('index'))
    return render_template('add_book.html')


if __name__ == '__main__':
    # Running the app this way might be problematic in the notebook environment.
    # This part is mainly for completeness of the app.py file.
    # To run the Flask app, you would typically execute app.py from a terminal.
    print("Flask app setup complete. 'app.py', and templates have been created/modified. To run, execute book_publishing_website/app.py from your terminal.")

Flask app setup complete. 'app.py', and templates have been created/modified. To run, execute book_publishing_website/app.py from your terminal.


##  search and filtering (optional)

### Subtask:
Implement search and filtering options to help users find books.


**Reasoning**:
Implement search and filtering for books by modifying the index route to accept query parameters, constructing a dynamic SQL query based on these parameters, executing the query, and updating the index.html template to include a search form.



In [ ]:
import sqlite3
import os
from flask import Flask, render_template, request, redirect, url_for, g

app = Flask(__name__)
DATABASE = 'book_publishing_website/book_publishing_website.db'

def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        db_dir = os.path.dirname(DATABASE)
        os.makedirs(db_dir, exist_ok=True)
        db = g._database = sqlite3.connect(DATABASE)
        db.row_factory = sqlite3.Row
    return db

@app.teardown_appcontext
def close_db(error):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

def init_db():
    with app.app_context():
        db = get_db()
        schema_file_path = os.path.join('book_publishing_website', 'schema.sql')
        with open(schema_file_path, mode='r') as f:
             db.cursor().executescript(f.read())
        db.commit()

schema_sql_content = """
DROP TABLE IF EXISTS books;
CREATE TABLE books (
    book_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    genre TEXT,
    publication_date TEXT,
    description TEXT,
    file_path TEXT,
    cover_image_path TEXT
);

INSERT INTO books (title, author, genre, publication_date, description, file_path, cover_image_path) VALUES
('The Great Novel', 'Famous Author', 'Fiction', '2023-01-15', 'A captivating story about life and adventure.', '/books/the_great_novel.pdf', '/static/covers/great_novel_cover.jpg'),
('Mystery of the Old House', 'Intriguing Writer', 'Mystery', '2022-11-20', 'A thrilling mystery set in a spooky old house.', '/books/mystery_old_house.epub', '/static/covers/mystery_house_cover.png'),
('Science of the Future', 'Dr. Innovator', 'Science', '2024-05-10', 'Exploring cutting-edge scientific advancements.', '/books/science_future.pdf', '/static/covers/science_cover.jpg');
"""

project_dir = 'book_publishing_website'
templates_dir = os.path.join(project_dir, 'templates')
os.makedirs(project_dir, exist_ok=True)
os.makedirs(templates_dir, exist_ok=True)

schema_file_path = os.path.join(project_dir, 'schema.sql')
with open(schema_file_path, 'w') as f:
    f.write(schema_sql_content)

# Update index.html to include a search form
index_html_content = """
<!doctype html>
<html>
<head>
    <title>Published Books</title>
</head>
<body>
    <h1>Published Books</h1>

    <form method="GET" action="{{ url_for('index') }}">
        <input type="text" name="title" placeholder="Search by Title" value="{{ request.args.get('title', '') }}">
        <input type="text" name="author" placeholder="Search by Author" value="{{ request.args.get('author', '') }}">
        <input type="text" name="genre" placeholder="Search by Genre" value="{{ request.args.get('genre', '') }}">
        <input type="submit" value="Search">
    </form>

    <p><a href="{{ url_for('add_book') }}">Add New Book</a></p>

    <ul>
    {% for book in books %}
        <li><a href="{{ url_for('book_detail', book_id=book['book_id']) }}">{{ book['title'] }} by {{ book['author'] }}</a></li>
    {% endfor %}
    </ul>
</body>
</html>
"""

index_file_path = os.path.join(templates_dir, 'index.html')
with open(index_file_path, 'w') as f:
    f.write(index_html_content)

book_detail_html_content = """
<!doctype html>
<html>
<head>
    <title>{{ book.title }}</title>
</head>
<body>
    <a href="{{ url_for('index') }}">Back to Book List</a>
    <h1>{{ book.title }}</h1>
    <p><strong>Author:</strong> {{ book.author }}</p>
    {% if book.genre %}
    <p><strong>Genre:</strong> {{ book.genre }}</p>
    {% endif %}
    {% if book.publication_date %}
    <p><strong>Publication Date:</strong> {{ book.publication_date }}</p>
    {% endif %}
    {% if book.description %}
    <p><strong>Description:</strong></p>
    <p>{{ book.description }}</p>
    {% endif %}
    {% if book.cover_image_path %}
    <p><img src="{{ url_for('static', filename=book.cover_image_path.split('/')[-1]) }}" alt="Cover Image" width="200"></p>
    {% endif %}
    {% if book.file_path %}
    <p><a href="{{ url_for('static', filename=book.file_path.split('/')[-1]) }}">Download Book</a></p>
    {% endif %}
</body>
</html>
"""
book_detail_file_path = os.path.join(templates_dir, 'book_detail.html')
with open(book_detail_file_path, 'w') as f:
    f.write(book_detail_html_content)

add_book_html_content = """
<!doctype html>
<html>
<head>
    <title>Add New Book</title>
</head>
<body>
    <h1>Add New Book</h1>
    <form method="POST" action="{{ url_for('add_book') }}">
        <label for="title">Title:</label><br>
        <input type="text" id="title" name="title" required><br><br>

        <label for="author">Author:</label><br>
        <input type="text" id="author" name="author" required><br><br>

        <label for="genre">Genre:</label><br>
        <input type="text" id="genre" name="genre"><br><br>

        <label for="publication_date">Publication Date:</label><br>
        <input type="text" id="publication_date" name="publication_date"><br><br>

        <label for="description">Description:</label><br>
        <textarea id="description" name="description"></textarea><br><br>

        <label for="file_path">File Path (e.g., /books/book.pdf):</label><br>
        <input type="text" id="file_path" name="file_path"><br><br>

        <label for="cover_image_path">Cover Image Path (e.g., /static/covers/cover.jpg):</label><br>
        <input type="text" id="cover_image_path" name="cover_image_path"><br><br>

        <input type="submit" value="Add Book">
    </form>
    <p><a href="{{ url_for('index') }}">Back to Book List</a></p>
</body>
</html>
"""
add_book_file_path = os.path.join(templates_dir, 'add_book.html')
with open(add_book_file_path, 'w') as f:
    f.write(add_book_html_content)


# Initialize the database with the schema and sample data
init_db()


@app.route('/')
def index():
    db = get_db()
    query = 'SELECT book_id, title, author FROM books'
    conditions = []
    params = []

    title_search = request.args.get('title')
    author_search = request.args.get('author')
    genre_search = request.args.get('genre')

    if title_search:
        conditions.append('title LIKE ?')
        params.append('%' + title_search + '%')
    if author_search:
        conditions.append('author LIKE ?')
        params.append('%' + author_search + '%')
    if genre_search:
        conditions.append('genre LIKE ?')
        params.append('%' + genre_search + '%')

    if conditions:
        query += ' WHERE ' + ' AND '.join(conditions)

    cursor = db.execute(query, params)
    books = cursor.fetchall()

    # Pass the search terms back to the template for pre-filling the form
    return render_template('index.html', books=books, request=request)

@app.route('/book/<int:book_id>')
def book_detail(book_id):
    db = get_db()
    cursor = db.execute('SELECT * FROM books WHERE book_id = ?', (book_id,))
    book = cursor.fetchone()
    if book is None:
        return "Book not found", 404
    return render_template('book_detail.html', book=book)

@app.route('/add_book', methods=('GET', 'POST'))
def add_book():
    if request.method == 'POST':
        title = request.form['title']
        author = request.form['author']
        genre = request.form.get('genre')
        publication_date = request.form.get('publication_date')
        description = request.form.get('description')
        file_path = request.form.get('file_path')
        cover_image_path = request.form.get('cover_image_path')

        db = get_db()
        db.execute(
            'INSERT INTO books (title, author, genre, publication_date, description, file_path, cover_image_path)'
            ' VALUES (?, ?, ?, ?, ?, ?, ?)',
            (title, author, genre, publication_date, description, file_path, cover_image_path)
        )
        db.commit()
        return redirect(url_for('index'))
    return render_template('add_book.html')


if __name__ == '__main__':
    # Running the app this way might be problematic in the notebook environment.
    # This part is mainly for completeness of the app.py file.
    # To run the Flask app, you would typically execute app.py from a terminal.
    print("Flask app setup complete. 'app.py', and templates have been created/modified. To run, execute book_publishing_website/app.py from your terminal.")

Flask app setup complete. 'app.py', and templates have been created/modified. To run, execute book_publishing_website/app.py from your terminal.


##  website's appearance

### Subtask:
Create templates and static files (CSS, JavaScript) to style the website.


**Reasoning**:
Create the necessary directories and files for static content (CSS and potentially JS) and add basic CSS styling rules to the stylesheet. Also, modify the existing HTML templates to link the stylesheet.



In [ ]:
import os
from flask import Flask, render_template, request, redirect, url_for, g

# Re-using the existing Flask app instance and database logic from previous steps
app = Flask(__name__)
DATABASE = 'book_publishing_website/book_publishing_website.db'

def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        db_dir = os.path.dirname(DATABASE)
        os.makedirs(db_dir, exist_ok=True)
        db = g._database = sqlite3.connect(DATABASE)
        db.row_factory = sqlite3.Row
    return db

@app.teardown_appcontext
def close_db(error):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

def init_db():
    with app.app_context():
        db = get_db()
        schema_file_path = os.path.join('book_publishing_website', 'schema.sql')
        with open(schema_file_path, mode='r') as f:
             db.cursor().executescript(f.read())
        db.commit()

# Re-create schema.sql with sample data
schema_sql_content = """
DROP TABLE IF EXISTS books;
CREATE TABLE books (
    book_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    genre TEXT,
    publication_date TEXT,
    description TEXT,
    file_path TEXT,
    cover_image_path TEXT
);

INSERT INTO books (title, author, genre, publication_date, description, file_path, cover_image_path) VALUES
('The Great Novel', 'Famous Author', 'Fiction', '2023-01-15', 'A captivating story about life and adventure.', '/books/the_great_novel.pdf', '/static/covers/great_novel_cover.jpg'),
('Mystery of the Old House', 'Intriguing Writer', 'Mystery', '2022-11-20', 'A thrilling mystery set in a spooky old house.', '/books/mystery_old_house.epub', '/static/covers/mystery_house_cover.png'),
('Science of the Future', 'Dr. Innovator', 'Science', '2024-05-10', 'Exploring cutting-edge scientific advancements.', '/books/science_future.pdf', '/static/covers/science_cover.jpg');
"""

project_dir = 'book_publishing_website'
templates_dir = os.path.join(project_dir, 'templates')
static_dir = os.path.join(project_dir, 'static')
css_dir = os.path.join(static_dir, 'css')
js_dir = os.path.join(static_dir, 'js') # Optional JS directory

# Create directories if they don't exist
os.makedirs(project_dir, exist_ok=True)
os.makedirs(templates_dir, exist_ok=True)
os.makedirs(static_dir, exist_ok=True)
os.makedirs(css_dir, exist_ok=True)
os.makedirs(js_dir, exist_ok=True) # Create JS directory

# Create schema.sql in the project directory
schema_file_path = os.path.join(project_dir, 'schema.sql')
with open(schema_file_path, 'w') as f:
    f.write(schema_sql_content)

# Create style.css in the css directory with basic styles
style_css_content = """
body {
    font-family: sans-serif;
    line-height: 1.6;
    margin: 20px;
    background-color: #f4f4f4;
    color: #333;
}

h1, h2 {
    color: #0056b3;
}

a {
    color: #007bff;
    text-decoration: none;
}

a:hover {
    text-decoration: underline;
}

ul {
    list-style: none;
    padding: 0;
}

li {
    background-color: #fff;
    margin-bottom: 10px;
    padding: 15px;
    border-radius: 5px;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
}

form {
    background-color: #fff;
    padding: 20px;
    border-radius: 5px;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
    margin-bottom: 20px;
}

form label {
    display: block;
    margin-bottom: 5px;
    font-weight: bold;
}

form input[type="text"],
form textarea {
    width: 100%;
    padding: 10px;
    margin-bottom: 10px;
    border: 1px solid #ddd;
    border-radius: 4px;
}

form input[type="submit"] {
    background-color: #007bff;
    color: white;
    padding: 10px 15px;
    border: none;
    border-radius: 4px;
    cursor: pointer;
    font-size: 1em;
}

form input[type="submit"]:hover {
    background-color: #0056b3;
}

img {
    max-width: 100%;
    height: auto;
}
"""
style_css_file_path = os.path.join(css_dir, 'style.css')
with open(style_css_file_path, 'w') as f:
    f.write(style_css_content)

# Create a basic script.js (optional)
script_js_content = """
// Basic JavaScript file
console.log("Script loaded!");
"""
script_js_file_path = os.path.join(js_dir, 'script.js')
with open(script_js_file_path, 'w') as f:
    f.write(script_js_content)


# Modify index.html to link to style.css
index_html_content = """
<!doctype html>
<html>
<head>
    <title>Published Books</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
</head>
<body>
    <h1>Published Books</h1>

    <form method="GET" action="{{ url_for('index') }}">
        <input type="text" name="title" placeholder="Search by Title" value="{{ request.args.get('title', '') }}">
        <input type="text" name="author" placeholder="Search by Author" value="{{ request.args.get('author', '') }}">
        <input type="text" name="genre" placeholder="Search by Genre" value="{{ request.args.get('genre', '') }}">
        <input type="submit" value="Search">
    </form>

    <p><a href="{{ url_for('add_book') }}">Add New Book</a></p>

    <ul>
    {% for book in books %}
        <li><a href="{{ url_for('book_detail', book_id=book['book_id']) }}">{{ book['title'] }} by {{ book['author'] }}</a></li>
    {% endfor %}
    </ul>
</body>
</html>
"""

index_file_path = os.path.join(templates_dir, 'index.html')
with open(index_file_path, 'w') as f:
    f.write(index_html_content)

# Modify book_detail.html to link to style.css
book_detail_html_content = """
<!doctype html>
<html>
<head>
    <title>{{ book.title }}</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
</head>
<body>
    <a href="{{ url_for('index') }}">Back to Book List</a>
    <h1>{{ book.title }}</h1>
    <p><strong>Author:</strong> {{ book.author }}</p>
    {% if book.genre %}
    <p><strong>Genre:</strong> {{ book.genre }}</p>
    {% endif %}
    {% if book.publication_date %}
    <p><strong>Publication Date:</strong> {{ book.publication_date }}</p>
    {% endif %}
    {% if book.description %}
    <p><strong>Description:</strong></p>
    <p>{{ book.description }}</p>
    {% endif %}
    {% if book.cover_image_path %}
    <p><img src="{{ url_for('static', filename=book.cover_image_path.split('/')[-1]) }}" alt="Cover Image" width="200"></p>
    {% endif %}
    {% if book.file_path %}
    <p><a href="{{ url_for('static', filename=book.file_path.split('/')[-1]) }}">Download Book</a></p>
    {% endif %}
</body>
</html>
"""
book_detail_file_path = os.path.join(templates_dir, 'book_detail.html')
with open(book_detail_file_path, 'w') as f:
    f.write(book_detail_html_content)

# Modify add_book.html to link to style.css
add_book_html_content = """
<!doctype html>
<html>
<head>
    <title>Add New Book</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
</head>
<body>
    <h1>Add New Book</h1>
    <form method="POST" action="{{ url_for('add_book') }}">
        <label for="title">Title:</label><br>
        <input type="text" id="title" name="title" required><br><br>

        <label for="author">Author:</label><br>
        <input type="text" id="author" name="author" required><br><br>

        <label for="genre">Genre:</label><br>
        <input type="text" id="genre" name="genre"><br><br>

        <label for="publication_date">Publication Date:</label><br>
        <input type="text" id="publication_date" name="publication_date"><br><br>

        <label for="description">Description:</label><br>
        <textarea id="description" name="description"></textarea><br><br>

        <label for="file_path">File Path (e.g., /books/book.pdf):</label><br>
        <input type="text" id="file_path" name="file_path"><br><br>

        <label for="cover_image_path">Cover Image Path (e.g., /static/covers/cover.jpg):</label><br>
        <input type="text" id="cover_image_path" name="cover_image_path"><br><br>

        <input type="submit" value="Add Book">
    </form>
    <p><a href="{{ url_for('index') }}">Back to Book List</a></p>
</body>
</html>
"""
add_book_file_path = os.path.join(templates_dir, 'add_book.html')
with open(add_book_file_path, 'w') as f:
    f.write(add_book_html_content)


# Initialize the database with the schema and sample data
init_db()

# Re-define Flask routes
@app.route('/')
def index():
    db = get_db()
    query = 'SELECT book_id, title, author FROM books'
    conditions = []
    params = []

    title_search = request.args.get('title')
    author_search = request.args.get('author')
    genre_search = request.args.get('genre')

    if title_search:
        conditions.append('title LIKE ?')
        params.append('%' + title_search + '%')
    if author_search:
        conditions.append('author LIKE ?')
        params.append('%' + author_search + '%')
    if genre_search:
        conditions.append('genre LIKE ?')
        params.append('%' + genre_search + '%')

    if conditions:
        query += ' WHERE ' + ' AND '.join(conditions)

    cursor = db.execute(query, params)
    books = cursor.fetchall()

    return render_template('index.html', books=books, request=request)

@app.route('/book/<int:book_id>')
def book_detail(book_id):
    db = get_db()
    cursor = db.execute('SELECT * FROM books WHERE book_id = ?', (book_id,))
    book = cursor.fetchone()
    if book is None:
        return "Book not found", 404
    return render_template('book_detail.html', book=book)

@app.route('/add_book', methods=('GET', 'POST'))
def add_book():
    if request.method == 'POST':
        title = request.form['title']
        author = request.form['author']
        genre = request.form.get('genre')
        publication_date = request.form.get('publication_date')
        description = request.form.get('description')
        file_path = request.form.get('file_path')
        cover_image_path = request.form.get('cover_image_path')

        db = get_db()
        db.execute(
            'INSERT INTO books (title, author, genre, publication_date, description, file_path, cover_image_path)'
            ' VALUES (?, ?, ?, ?, ?, ?, ?)',
            (title, author, genre, publication_date, description, file_path, cover_image_path)
        )
        db.commit()
        return redirect(url_for('index'))
    return render_template('add_book.html')


if __name__ == '__main__':
    # Running the app this way might be problematic in the notebook environment.
    # This part is mainly for completeness of the app.py file.
    # To run the Flask app, you would typically execute app.py from a terminal.
    print("Flask app setup complete. Static files and updated templates created. To run, execute book_publishing_website/app.py from your terminal.")


Flask app setup complete. Static files and updated templates created. To run, execute book_publishing_website/app.py from your terminal.


## Deploy the website

### Subtask:
Choose a hosting platform and deploy the web application.


**Reasoning**:
Identify potential hosting platforms, evaluate them based on ease of use, cost, and suitability for a Flask app with SQLite, and prepare the application for deployment by creating `requirements.txt` and `Procfile`. This addresses instructions 1, 2, and 3.



In [ ]:
import os

# Step 1 & 2: Identify and evaluate potential hosting platforms
# Heroku: Easy to use, free tier available (though with limitations), supports Flask and SQLite (though SQLite on ephemeral filesystems like Heroku's free tier has limitations for persistent data).
# PythonAnywhere: Designed for Python web apps, offers a free tier, supports Flask and persistent SQLite databases directly. Very easy to use for simple apps.
# Render: Free tier available, supports web services and persistent disks (for SQLite), good for Flask.
# Vercel: Primarily for serverless functions and frontend hosting, not ideal for a traditional Flask app with SQLite database file.

# PythonAnywhere seems like a good balance of ease of use, free tier availability, and suitability
# for a Flask app with a persistent SQLite database file for a small project.

chosen_platform = "PythonAnywhere"
print(f"Chosen hosting platform: {chosen_platform}")

# Step 3: Prepare the Flask application for deployment

# Create requirements.txt
requirements_content = """
Flask
sqlite3
"""
requirements_file_path = 'book_publishing_website/requirements.txt'
with open(requirements_file_path, 'w') as f:
    f.write(requirements_content.strip())

print(f"Created {requirements_file_path}")

# PythonAnywhere uses a web-based configuration, not a Procfile.
# The entry point is configured in the PythonAnywhere web interface.
# For other platforms like Heroku or Render, a Procfile would look like:
# web: gunicorn app:app

# Since we chose PythonAnywhere, we don't need a Procfile.

print("Preparation for deployment complete. requirements.txt created. No Procfile needed for PythonAnywhere.")


Chosen hosting platform: PythonAnywhere
Created book_publishing_website/requirements.txt
Preparation for deployment complete. requirements.txt created. No Procfile needed for PythonAnywhere.


## Summary:

### Q&A
*   **What web framework was chosen?** Flask was chosen as the web framework for the project.
*   **Is user authentication implemented?** No, user authentication is not implemented in the initial version, as it was deemed optional for a small website.
*   **What database is used?** SQLite is used for the database.
*   **Is there search functionality?** Yes, search functionality by title, author, and genre has been implemented.
*   **Was the website successfully deployed?** No, the deployment could not be fully completed within the notebook environment as it requires manual steps on a hosting platform.

### Data Analysis Key Findings
*   Flask was selected as the web framework over Django due to its lightweight nature and suitability for incremental feature additions in a small project.
*   The project directory structure (`book_publishing_website` with `app.py`, `templates`, and `static` subdirectories) was successfully created.
*   A database schema for a `books` table, including fields for title, author, genre, publication date, description, and file/cover image paths, was designed using SQLite. A `users` table was planned for potential future user authentication.
*   Functionality to list published books was implemented, querying the database and displaying the title and author on the index page.
*   A book detail page was created, allowing users to view comprehensive information about a specific book by clicking on its title from the list.
*   Functionality to add new books was implemented, including a form for input and database insertion logic.
*   Search and filtering by title, author, and genre were added to the book listing page, dynamically querying the database based on user input.
*   Basic styling using CSS was added by creating a `style.css` file and linking it to the HTML templates.
*   Deployment preparation included creating a `requirements.txt` file.
*   Full website deployment to a hosting platform (PythonAnywhere) was not achievable within the execution environment, requiring manual steps by the user.




## Prepare the application for google cloud deployment

### Subtask:
Modify the application code and add necessary configuration files for deployment on Google Cloud.


**Reasoning**:
Update `requirements.txt` to include Gunicorn and create a `Procfile` for Google Cloud deployment preparation.



In [ ]:
import sqlite3
import os
from flask import Flask, render_template, request, redirect, url_for, g

# Check if app instance already exists to avoid re-creating and re-adding routes
if 'app' not in globals():
    app = Flask(__name__)

project_dir = 'book_publishing_website'
requirements_file_path = os.path.join(project_dir, 'requirements.txt')
procfile_path = os.path.join(project_dir, 'Procfile')
app_file_path = os.path.join(project_dir, 'app.py')

# Instruction 1: Update requirements.txt to include Gunicorn
requirements_content = """
Flask
gunicorn
"""
with open(requirements_file_path, 'w') as f:
    f.write(requirements_content.strip())

print(f"Updated {requirements_file_path} to include Gunicorn.")

# Instruction 2: Create a Procfile
# The command assumes your Flask app instance is named 'app' in app.py
procfile_content = "web: gunicorn app:app --bind 0.0.0.0:$PORT"
with open(procfile_path, 'w') as f:
    f.write(procfile_content.strip())

print(f"Created {procfile_path}")

# Instruction 3: Ensure app.py is runnable by Gunicorn
# The existing app.py structure with 'app = Flask(__name__)' is compatible with Gunicorn.
# No changes needed to app.py for this instruction.

# Instruction 4: Review database connection logic
# The current SQLite setup is simple. For a production Google Cloud deployment,
# this would typically be replaced with a managed database service like Cloud SQL (PostgreSQL or MySQL).
# The connection string/method would need to be updated to use database credentials
# and potentially environment variables. For now, the existing logic is sufficient
# for the preparation steps, but this is noted as a required change for actual deployment.
print("Database connection logic reviewed. Note that SQLite needs to be replaced with a production database for actual Google Cloud deployment.")

# Instruction 5: Add environment variables or configuration settings
# For this basic app, no specific environment variables are strictly required yet.
# Secrets (like database credentials, if using a production DB) should be handled
# securely using environment variables or Google Cloud Secret Manager during actual deployment.
print("Environment variables and configuration settings considered. Secure handling of secrets is planned for actual deployment.")

# Re-create schema.sql with sample data to ensure it exists after potential previous runs
schema_sql_content = """
DROP TABLE IF EXISTS books;
CREATE TABLE books (
    book_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    genre TEXT,
    publication_date TEXT,
    description TEXT,
    file_path TEXT,
    cover_image_path TEXT
);

INSERT INTO books (title, author, genre, publication_date, description, file_path, cover_image_path) VALUES
('The Great Novel', 'Famous Author', 'Fiction', '2023-01-15', 'A captivating story about life and adventure.', '/books/the_great_novel.pdf', '/static/covers/great_novel_cover.jpg'),
('Mystery of the Old House', 'Intriguing Writer', 'Mystery', '2022-11-20', 'A thrilling mystery set in a spooky old house.', '/books/mystery_old_house.epub', '/static/covers/mystery_house_cover.png'),
('Science of the Future', 'Dr. Innovator', 'Science', '2024-05-10', 'Exploring cutting-edge scientific advancements.', '/books/science_future.pdf', '/static/covers/science_cover.jpg');
"""
schema_file_path = os.path.join(project_dir, 'schema.sql')
with open(schema_file_path, 'w') as f:
    f.write(schema_sql_content)
print(f"Re-created {schema_file_path}")

# Ensure templates directory exists
templates_dir = os.path.join(project_dir, 'templates')
os.makedirs(templates_dir, exist_ok=True)

# Re-create index.html to ensure it exists
index_html_content = """
<!doctype html>
<html>
<head>
    <title>Published Books</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
</head>
<body>
    <h1>Published Books</h1>

    <form method="GET" action="{{ url_for('index') }}">
        <input type="text" name="title" placeholder="Search by Title" value="{{ request.args.get('title', '') }}">
        <input type="text" name="author" placeholder="Search by Author" value="{{ request.args.get('author', '') }}">
        <input type="text" name="genre" placeholder="Search by Genre" value="{{ request.args.get('genre', '') }}">
        <input type="submit" value="Search">
    </form>

    <p><a href="{{ url_for('add_book') }}">Add New Book</a></p>

    <ul>
    {% for book in books %}
        <li><a href="{{ url_for('book_detail', book_id=book['book_id']) }}">{{ book['title'] }} by {{ book['author'] }}</a></li>
    {% endfor %}
    </ul>
</body>
</html>
"""
index_file_path = os.path.join(templates_dir, 'index.html')
with open(index_file_path, 'w') as f:
    f.write(index_html_content)
print(f"Re-created {index_file_path}")

# Re-create book_detail.html to ensure it exists
book_detail_html_content = """
<!doctype html>
<html>
<head>
    <title>{{ book.title }}</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
</head>
<body>
    <a href="{{ url_for('index') }}">Back to Book List</a>
    <h1>{{ book.title }}</h1>
    <p><strong>Author:</strong> {{ book.author }}</p>
    {% if book.genre %}
    <p><strong>Genre:</strong> {{ book.genre }}</p>
    {% endif %}
    {% if book.publication_date %}
    <p><strong>Publication Date:</strong> {{ book.publication_date }}</p>
    {% endif %}
    {% if book.description %}
    <p><strong>Description:</strong></p>
    <p>{{ book.description }}</p>
    {% endif %}
    {% if book.cover_image_path %}
    <p><img src="{{ url_for('static', filename=book.cover_image_path.split('/')[-1]) }}" alt="Cover Image" width="200"></p>
    {% endif %}
    {% if book.file_path %}
    <p><a href="{{ url_for('static', filename=book.file_path.split('/')[-1]) }}">Download Book</a></p>
    {% endif %}
</body>
</html>
"""
book_detail_file_path = os.path.join(templates_dir, 'book_detail.html')
with open(book_detail_file_path, 'w') as f:
    f.write(book_detail_html_content)
print(f"Re-created {book_detail_file_path}")

# Re-create add_book.html to ensure it exists
add_book_html_content = """
<!doctype html>
<html>
<head>
    <title>Add New Book</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
</head>
<body>
    <h1>Add New Book</h1>
    <form method="POST" action="{{ url_for('add_book') }}">
        <label for="title">Title:</label><br>
        <input type="text" id="title" name="title" required><br><br>

        <label for="author">Author:</label><br>
        <input type="text" id="author" name="author" required><br><br>

        <label for="genre">Genre:</label><br>
        <input type="text" id="genre" name="genre"><br><br>

        <label for="publication_date">Publication Date:</label><br>
        <input type="text" id="publication_date" name="publication_date"><br><br>

        <label for="description">Description:</label><br>
        <textarea id="description" name="description"></textarea><br><br>

        <label for="file_path">File Path (e.g., /books/book.pdf):</label><br>
        <input type="text" id="file_path" name="file_path"><br><br>

        <label for="cover_image_path">Cover Image Path (e.g., /static/covers/cover.jpg):</label><br>
        <input type="text" id="cover_image_path" name="cover_image_path"><br><br>

        <input type="submit" value="Add Book">
    </form>
    <p><a href="{{ url_for('index') }}">Back to Book List</a></p>
</body>
</html>
"""
add_book_file_path = os.path.join(templates_dir, 'add_book.html')
with open(add_book_file_path, 'w') as f:
    f.write(add_book_html_content)
print(f"Re-created {add_book_file_path}")

# Ensure static directories and files exist
static_dir = os.path.join(project_dir, 'static')
css_dir = os.path.join(static_dir, 'css')
js_dir = os.path.join(static_dir, 'js') # Optional JS directory

os.makedirs(static_dir, exist_ok=True)
os.makedirs(css_dir, exist_ok=True)
os.makedirs(js_dir, exist_ok=True) # Create JS directory

# Create style.css in the css directory with basic styles
style_css_content = """
body {
    font-family: sans-serif;
    line-height: 1.6;
    margin: 20px;
    background-color: #f4f4f4;
    color: #333;
}

h1, h2 {
    color: #0056b3;
}

a {
    color: #007bff;
    text-decoration: none;
}

a:hover {
    text-decoration: underline;
}

ul {
    list-style: none;
    padding: 0;
}

li {
    background-color: #fff;
    margin-bottom: 10px;
    padding: 15px;
    border-radius: 5px;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
}

form {
    background-color: #fff;
    padding: 20px;
    border-radius: 5px;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
    margin-bottom: 20px;
}

form label {
    display: block;
    margin-bottom: 5px;
    font-weight: bold;
}

form input[type="text"],
form textarea {
    width: 100%;
    padding: 10px;
    margin-bottom: 10px;
    border: 1px solid #ddd;
    border-radius: 4px;
}

form input[type="submit"] {
    background-color: #007bff;
    color: white;
    padding: 10px 15px;
    border: none;
    border-radius: 4px;
    cursor: pointer;
    font-size: 1em;
}

form input[type="submit"]:hover {
    background-color: #0056b3;
}

img {
    max-width: 100%;
    height: auto;
}
"""
style_css_file_path = os.path.join(css_dir, 'style.css')
with open(style_css_file_path, 'w') as f:
    f.write(style_css_content)
print(f"Re-created {style_css_file_path}")


# Create a basic script.js (optional)
script_js_content = """
// Basic JavaScript file
console.log("Script loaded!");
"""
script_js_file_path = os.path.join(js_dir, 'script.js')
with open(script_js_file_path, 'w') as f:
    f.write(script_js_content)
print(f"Re-created {script_js_file_path}")

# Initialize the database with the schema and sample data
DATABASE = 'book_publishing_website/book_publishing_website.db'
def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        db_dir = os.path.dirname(DATABASE)
        os.makedirs(db_dir, exist_ok=True)
        db = g._database = sqlite3.connect(DATABASE)
        db.row_factory = sqlite3.Row
    return db

@app.teardown_appcontext
def close_db(error):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

def init_db():
    with app.app_context():
        db = get_db()
        schema_file_path = os.path.join('book_publishing_website', 'schema.sql')
        with open(schema_file_path, mode='r') as f:
             db.cursor().executescript(f.read())
        db.commit()

init_db()
print("Database initialized with schema and sample data.")

# Check if the routes are already defined before adding them
if '/' not in [r.rule for r in app.url_map.iter_rules()]:
    @app.route('/')
    def index():
        db = get_db()
        query = 'SELECT book_id, title, author FROM books'
        conditions = []
        params = []

        title_search = request.args.get('title')
        author_search = request.args.get('author')
        genre_search = request.args.get('genre')

        if title_search:
            conditions.append('title LIKE ?')
            params.append('%' + title_search + '%')
        if author_search:
            conditions.append('author LIKE ?')
            params.append('%' + author_search + '%')
        if genre_search:
            conditions.append('genre LIKE ?')
            params.append('%' + genre_search + '%')

        if conditions:
            query += ' WHERE ' + ' AND '.join(conditions)

        cursor = db.execute(query, params)
        books = cursor.fetchall()

        return render_template('index.html', books=books, request=request)

if '/book/<int:book_id>' not in [r.rule for r in app.url_map.iter_rules()]:
    @app.route('/book/<int:book_id>')
    def book_detail(book_id):
        db = get_db()
        cursor = db.execute('SELECT * FROM books WHERE book_id = ?', (book_id,))
        book = cursor.fetchone()
        if book is None:
            return "Book not found", 404
        return render_template('book_detail.html', book=book)

if '/add_book' not in [r.rule for r in app.url_map.iter_rules()]:
    @app.route('/add_book', methods=('GET', 'POST'))
    def add_book():
        if request.method == 'POST':
            title = request.form['title']
            author = request.form['author']
            genre = request.form.get('genre')
            publication_date = request.form.get('publication_date')
            description = request.form.get('description')
            file_path = request.form.get('file_path')
            cover_image_path = request.form.get('cover_image_path')

            db = get_db()
            db.execute(
                'INSERT INTO books (title, author, genre, publication_date, description, file_path, cover_image_path)'
                ' VALUES (?, ?, ?, ?, ?, ?, ?)',
                (title, author, genre, publication_date, description, file_path, cover_image_path)
            )
            db.commit()
            return redirect(url_for('index'))
        return render_template('add_book.html')

print("Flask routes defined/ensured.")

Updated book_publishing_website/requirements.txt to include Gunicorn.
Created book_publishing_website/Procfile
Database connection logic reviewed. Note that SQLite needs to be replaced with a production database for actual Google Cloud deployment.
Environment variables and configuration settings considered. Secure handling of secrets is planned for actual deployment.
Re-created book_publishing_website/schema.sql
Re-created book_publishing_website/templates/index.html
Re-created book_publishing_website/templates/book_detail.html
Re-created book_publishing_website/templates/add_book.html
Re-created book_publishing_website/static/css/style.css
Re-created book_publishing_website/static/js/script.js
Database initialized with schema and sample data.
Flask routes defined/ensured.


##  google cloud deployment option

### Subtask:
 suitable Google Cloud service for hosting the Flask application.


**Reasoning**:
Identify potential Google Cloud hosting services, evaluate them based on the specified criteria, and choose the most suitable option for the Flask application with SQLite.



In [ ]:
# Step 1: Consider the requirements
# The application is small-scale, uses Flask, and initially relies on a file-based database (SQLite).
# It has potential for future growth (e.g., user authentication, more features).
# Persistence of the database file is crucial for the SQLite approach.

# Step 2: Evaluate suitable Google Cloud services

# Option 1: App Engine Standard
# - Ease of deployment: Relatively easy for Python apps.
# - Cost: Has a free tier, scales to zero.
# - Scalability: Highly scalable, automatic scaling.
# - Suitability for Python/Flask: Good support for Flask.
# - Suitability for SQLite: Poor. App Engine Standard's filesystem is ephemeral, meaning
#   the SQLite database file would be lost when instances restart or scale down.
#   Requires using a persistent database service like Cloud SQL.

# Option 2: App Engine Flexible
# - Ease of deployment: Requires more configuration than Standard, uses Docker.
# - Cost: More expensive than Standard, generally doesn't scale to zero.
# - Scalability: Automatic scaling, but less granular than Standard.
# - Suitability for Python/Flask: Good support.
# - Suitability for SQLite: Possible if using persistent disks, but adds complexity.
#   Still better suited for managed databases like Cloud SQL.

# Option 3: Cloud Run
# - Ease of deployment: Very easy, container-based (can deploy from source or Dockerfile).
# - Cost: Pay-per-use, scales to zero.
# - Scalability: Scales automatically based on requests.
# - Suitability for Python/Flask: Excellent, designed for stateless web services.
# - Suitability for SQLite: Poor for persistent data. Similar to App Engine Standard, the container's filesystem
#   is ephemeral. Persistent storage requires mounting Cloud Storage or using a managed database.

# Option 4: Compute Engine (GCE)
# - Ease of deployment: Requires manual setup of a VM, web server (like Nginx/Apache),
#   and application environment. Most complex option.
# - Cost: Pay for VM instance time, can be cost-effective for constant load but less so for variable traffic.
# - Scalability: Manual scaling (adding/removing VMs) or using Instance Groups.
# - Suitability for Python/Flask: Full control, can run any Python app.
# - Suitability for SQLite: Good. You can attach a persistent disk to the VM and store the SQLite database file there.

# Considering the initial requirement of using SQLite and the need for persistent storage
# for the database file, App Engine Standard, App Engine Flexible (without persistent disk),
# and Cloud Run are not ideal due to their ephemeral filesystems.
# Compute Engine allows for persistent storage with SQLite, but it is the most complex to manage and deploy.

# However, for a production application on Google Cloud, using a managed database service
# like Cloud SQL (PostgreSQL or MySQL) is strongly recommended over a file-based SQLite database,
# regardless of the hosting service chosen. Managed databases offer better scalability, reliability,
# backups, and security.

# Reconsidering the options, if the goal is the easiest deployment of a Flask app on Google Cloud,
# even if it means planning to migrate from SQLite to Cloud SQL later, Cloud Run is an excellent choice
# due to its simplicity, scalability to zero (cost-saving for a small/low-traffic site),
# and ease of use for containerized applications. The SQLite database would be a limitation in production
# on Cloud Run and would need to be replaced.

# If the *absolute requirement* is to use SQLite and have persistent storage on Google Cloud,
# then Compute Engine with a persistent disk is the most direct, albeit more complex, option.

# Let's assume the plan is to eventually migrate to Cloud SQL, making the ephemeral filesystem
# of Cloud Run acceptable for the initial deployment phase as a learning exercise or for testing
# the application logic (excluding persistent data).

# Step 3: Choose the most appropriate service and explain reasoning

chosen_google_cloud_service = "Cloud Run"

reasoning = """
Cloud Run is chosen for hosting the Flask application on Google Cloud Platform.
Reasoning:
1.  **Ease of Deployment:** Cloud Run offers a simple and fast deployment process, especially when deploying directly from source code or a container image. This aligns well with the goal of quickly getting a small Flask app online.
2.  **Scalability and Cost:** Cloud Run scales automatically based on request traffic and scales to zero when there is no traffic. This makes it very cost-effective for a small website with potentially low initial usage.
3.  **Suitability for Python/Flask:** Cloud Run is well-suited for hosting stateless web applications built with frameworks like Flask.
4.  **Future-Proofing (with database consideration):** While Cloud Run's ephemeral filesystem is not ideal for the current SQLite database setup (as the data would not persist across container instances/restarts), the recommended path for production Flask apps on Google Cloud is to use a managed database like Cloud SQL. Cloud Run integrates well with Cloud SQL, making the future transition to a persistent database straightforward. The initial deployment can serve as a demonstration of the application logic, with the understanding that the database layer needs to be updated for production persistence.
"""

print(f"Chosen Google Cloud service: {chosen_google_cloud_service}")
print(reasoning)

Chosen Google Cloud service: Cloud Run

Cloud Run is chosen for hosting the Flask application on Google Cloud Platform.
Reasoning:
1.  **Ease of Deployment:** Cloud Run offers a simple and fast deployment process, especially when deploying directly from source code or a container image. This aligns well with the goal of quickly getting a small Flask app online.
2.  **Scalability and Cost:** Cloud Run scales automatically based on request traffic and scales to zero when there is no traffic. This makes it very cost-effective for a small website with potentially low initial usage.
3.  **Suitability for Python/Flask:** Cloud Run is well-suited for hosting stateless web applications built with frameworks like Flask.
4.  **Future-Proofing (with database consideration):** While Cloud Run's ephemeral filesystem is not ideal for the current SQLite database setup (as the data would not persist across container instances/restarts), the recommended path for production Flask apps on Google Cloud is

## Containerize the application (optional but recommended)

### Subtask:
Create a Dockerfile to containerize the Flask application for easier deployment.


**Reasoning**:
Create the Dockerfile in the project directory and add the specified content to containerize the Flask application.



In [ ]:
import os

project_dir = 'book_publishing_website'
dockerfile_path = os.path.join(project_dir, 'Dockerfile')

# Instruction 2-8: Define the Dockerfile content
dockerfile_content = """
# Use a base Python image
FROM python:3.9-slim

# Set the working directory
WORKDIR /app

# Copy the requirements file and install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy the rest of the application code
COPY . .

# Expose the port the app runs on (Cloud Run defaults to 8080)
EXPOSE 8080

# Run the application using Gunicorn, binding to 0.0.0.0 and the PORT environment variable
CMD ["gunicorn", "app:app", "--bind", "0.0.0.0:$PORT"]
"""

# Create the Dockerfile
with open(dockerfile_path, 'w') as f:
    f.write(dockerfile_content.strip())

print(f"Created {dockerfile_path}")

# Ensure necessary files for the Docker build exist
# (requirements.txt, app.py, templates, static, schema.sql should already exist from previous steps)
# We will explicitly ensure requirements.txt and app.py exist for robustness.
requirements_content = """
Flask
gunicorn
"""
requirements_file_path = os.path.join(project_dir, 'requirements.txt')
with open(requirements_file_path, 'w') as f:
    f.write(requirements_content.strip())
print(f"Ensured {requirements_file_path} exists.")

app_file_content = """
import sqlite3
import os
from flask import Flask, render_template, request, redirect, url_for, g

app = Flask(__name__)
DATABASE = 'book_publishing_website.db' # Database file inside the container's /app directory

def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        # The database file will be created directly in the WORKDIR (/app)
        db = g._database = sqlite3.connect(DATABASE)
        db.row_factory = sqlite3.Row
    return db

@app.teardown_appcontext
def close_db(error):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

# Function to initialize the database (create table and add some sample data)
# This will run once when the container is built or on the first request if not pre-initialized
def init_db():
    # Check if the database file already exists to avoid re-initializing
    if not os.path.exists(DATABASE):
         with app.app_context():
            db = get_db()
            # Use the schema.sql file from the copied content
            schema_file_path = 'schema.sql' # schema.sql is copied to /app
            with open(schema_file_path, mode='r') as f:
                 db.cursor().executescript(f.read())
            db.commit()
            print("Database initialized.")
    else:
        print("Database already exists.")


# Initialize the database when the application starts
# Note: For Cloud Run with ephemeral storage, this will re-initialize on each new instance.
# A persistent database (like Cloud SQL) is needed for production.
init_db()


@app.route('/')
def index():
    db = get_db()
    query = 'SELECT book_id, title, author FROM books'
    conditions = []
    params = []

    title_search = request.args.get('title')
    author_search = request.args.get('author')
    genre_search = request.args.get('genre')

    if title_search:
        conditions.append('title LIKE ?')
        params.append('%' + title_search + '%')
    if author_search:
        conditions.append('author LIKE ?')
        params.append('%' + author_search + '%')
    if genre_search:
        conditions.append('genre LIKE ?')
        params.append('%' + genre_search + '%')

    if conditions:
        query += ' WHERE ' + ' AND '.join(conditions)

    cursor = db.execute(query, params)
    books = cursor.fetchall()

    return render_template('index.html', books=books, request=request)

@app.route('/book/<int:book_id>')
def book_detail(book_id):
    db = get_db()
    cursor = db.execute('SELECT * FROM books WHERE book_id = ?', (book_id,))
    book = cursor.fetchone()
    if book is None:
        return "Book not found", 404
    return render_template('book_detail.html', book=book)

@app.route('/add_book', methods=('GET', 'POST'))
def add_book():
    if request.method == 'POST':
        title = request.form['title']
        author = request.form['author']
        genre = request.form.get('genre')
        publication_date = request.form.get('publication_date')
        description = request.form.get('description')
        file_path = request.form.get('file_path')
        cover_image_path = request.form.get('cover_image_path')

        db = get_db()
        db.execute(
            'INSERT INTO books (title, author, genre, publication_date, description, file_path, cover_image_path)'
            ' VALUES (?, ?, ?, ?, ?, ?, ?)',
            (title, author, genre, publication_date, description, file_path, cover_image_path)
        )
        db.commit()
        return redirect(url_for('index'))
    return render_template('add_book.html')


# This __main__ block is typically used for local development, not for Gunicorn in production.
# The CMD in the Dockerfile tells Gunicorn how to run the app.
# if __name__ == '__main__':
#     app.run(debug=True)

"""
app_file_path = os.path.join(project_dir, 'app.py')
with open(app_file_path, 'w') as f:
    f.write(app_file_content.strip())
print(f"Ensured {app_file_path} exists and updated database path for container.")

# Ensure schema.sql exists
schema_sql_content = """
DROP TABLE IF EXISTS books;
CREATE TABLE books (
    book_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    genre TEXT,
    publication_date TEXT,
    description TEXT,
    file_path TEXT,
    cover_image_path TEXT
);

INSERT INTO books (title, author, genre, publication_date, description, file_path, cover_image_path) VALUES
('The Great Novel', 'Famous Author', 'Fiction', '2023-01-15', 'A captivating story about life and adventure.', '/books/the_great_novel.pdf', '/static/covers/great_novel_cover.jpg'),
('Mystery of the Old House', 'Intriguing Writer', 'Mystery', '2022-11-20', 'A thrilling mystery set in a spooky old house.', '/books/mystery_old_house.epub', '/static/covers/mystery_house_cover.png'),
('Science of the Future', 'Dr. Innovator', 'Science', '2024-05-10', 'Exploring cutting-edge scientific advancements.', '/books/science_future.pdf', '/static/covers/science_cover.jpg');
"""
schema_file_path = os.path.join(project_dir, 'schema.sql')
with open(schema_file_path, 'w') as f:
    f.write(schema_sql_content)
print(f"Ensured {schema_file_path} exists.")

# Ensure templates directory and content exist
templates_dir = os.path.join(project_dir, 'templates')
os.makedirs(templates_dir, exist_ok=True)
index_html_content = """
<!doctype html>
<html>
<head>
    <title>Published Books</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
</head>
<body>
    <h1>Published Books</h1>

    <form method="GET" action="{{ url_for('index') }}">
        <input type="text" name="title" placeholder="Search by Title" value="{{ request.args.get('title', '') }}">
        <input type="text" name="author" placeholder="Search by Author" value="{{ request.args.get('author', '') }}">
        <input type="text" name="genre" placeholder="Search by Genre" value="{{ request.args.get('genre', '') }}">
        <input type="submit" value="Search">
    </form>

    <p><a href="{{ url_for('add_book') }}">Add New Book</a></p>

    <ul>
    {% for book in books %}
        <li><a href="{{ url_for('book_detail', book_id=book['book_id']) }}">{{ book['title'] }} by {{ book['author'] }}</a></li>
    {% endfor %}
    </ul>
</body>
</html>
"""
index_file_path = os.path.join(templates_dir, 'index.html')
with open(index_file_path, 'w') as f:
    f.write(index_html_content)
print(f"Ensured {index_file_path} exists.")

book_detail_html_content = """
<!doctype html>
<html>
<head>
    <title>{{ book.title }}</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
</head>
<body>
    <a href="{{ url_for('index') }}">Back to Book List</a>
    <h1>{{ book.title }}</h1>
    <p><strong>Author:</strong> {{ book.author }}</p>
    {% if book.genre %}
    <p><strong>Genre:</strong> {{ book.genre }}</p>
    {% endif %}
    {% if book.publication_date %}
    <p><strong>Publication Date:</strong> {{ book.publication_date }}</p>
    {% endif %}
    {% if book.description %}
    <p><strong>Description:</strong></p>
    <p>{{ book.description }}</p>
    {% endif %}
    {% if book.cover_image_path %}
    <p><img src="{{ url_for('static', filename=book.cover_image_path.split('/')[-1]) }}" alt="Cover Image" width="200"></p>
    {% endif %}
    {% if book.file_path %}
    <p><a href="{{ url_for('static', filename=book.file_path.split('/')[-1]) }}">Download Book</a></p>
    {% endif %}
</body>
</html>
"""
book_detail_file_path = os.path.join(templates_dir, 'book_detail.html')
with open(book_detail_file_path, 'w') as f:
    f.write(book_detail_html_content)
print(f"Ensured {book_detail_file_path} exists.")

add_book_html_content = """
<!doctype html>
<html>
<head>
    <title>Add New Book</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
</head>
<body>
    <h1>Add New Book</h1>
    <form method="POST" action="{{ url_for('add_book') }}">
        <label for="title">Title:</label><br>
        <input type="text" id="title" name="title" required><br><br>

        <label for="author">Author:</label><br>
        <input type="text" id="author" name="author" required><br><br>

        <label for="genre">Genre:</label><br>
        <input type="text" id="genre" name="genre"><br><br>

        <label for="publication_date">Publication Date:</label><br>
        <input type="text" id="publication_date" name="publication_date"><br><br>

        <label for="description">Description:</label><br>
        <textarea id="description" name="description"></textarea><br><br>

        <label for="file_path">File Path (e.g., /books/book.pdf):</label><br>
        <input type="text" id="file_path" name="file_path"><br><br>

        <label for="cover_image_path">Cover Image Path (e.g., /static/covers/cover.jpg):</label><br>
        <input type="text" id="cover_image_path" name="cover_image_path"><br><br>

        <input type="submit" value="Add Book">
    </form>
    <p><a href="{{ url_for('index') }}">Back to Book List</a></p>
</body>
</html>
"""
add_book_file_path = os.path.join(templates_dir, 'add_book.html')
with open(add_book_file_path, 'w') as f:
    f.write(add_book_html_content)
print(f"Ensured {add_book_file_path} exists.")


# Ensure static directories and files exist
static_dir = os.path.join(project_dir, 'static')
css_dir = os.path.join(static_dir, 'css')
js_dir = os.path.join(static_dir, 'js') # Optional JS directory

os.makedirs(static_dir, exist_ok=True)
os.makedirs(css_dir, exist_ok=True)
os.makedirs(js_dir, exist_ok=True) # Create JS directory

# Create style.css in the css directory with basic styles
style_css_content = """
body {
    font-family: sans-serif;
    line-height: 1.6;
    margin: 20px;
    background-color: #f4f4f4;
    color: #333;
}

h1, h2 {
    color: #0056b3;
}

a {
    color: #007bff;
    text-decoration: none;
}

a:hover {
    text-decoration: underline;
}

ul {
    list-style: none;
    padding: 0;
}

li {
    background-color: #fff;
    margin-bottom: 10px;
    padding: 15px;
    border-radius: 5px;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
}

form {
    background-color: #fff;
    padding: 20px;
    border-radius: 5px;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
    margin-bottom: 20px;
}

form label {
    display: block;
    margin-bottom: 5px;
    font-weight: bold;
}

form input[type="text"],
form textarea {
    width: 100%;
    padding: 10px;
    margin-bottom: 10px;
    border: 1px solid #ddd;
    border-radius: 4px;
}

form input[type="submit"] {
    background-color: #007bff;
    color: white;
    padding: 10px 15px;
    border: none;
    border-radius: 4px;
    cursor: pointer;
    font-size: 1em;
}

form input[type="submit"]:hover {
    background-color: #0056b3;
}

img {
    max-width: 100%;
    height: auto;
}
"""
style_css_file_path = os.path.join(css_dir, 'style.css')
with open(style_css_file_path, 'w') as f:
    f.write(style_css_content)
print(f"Ensured {style_css_file_path} exists.")


# Create a basic script.js (optional)
script_js_content = """
// Basic JavaScript file
console.log("Script loaded!");
"""
script_js_file_path = os.path.join(js_dir, 'script.js')
with open(script_js_file_path, 'w') as f:
    f.write(script_js_content)
print(f"Ensured {script_js_file_path} exists.")

Created book_publishing_website/Dockerfile
Ensured book_publishing_website/requirements.txt exists.
Ensured book_publishing_website/app.py exists and updated database path for container.
Ensured book_publishing_website/schema.sql exists.
Ensured book_publishing_website/templates/index.html exists.
Ensured book_publishing_website/templates/book_detail.html exists.
Ensured book_publishing_website/templates/add_book.html exists.
Ensured book_publishing_website/static/css/style.css exists.
Ensured book_publishing_website/static/js/script.js exists.


## Containerize the application (optional but recommended)

### Subtask:
Create a Dockerfile to containerize the Flask application for easier deployment.

**Reasoning**:
Create the Dockerfile in the project directory and add the specified content to containerize the Flask application.

In [ ]:
import os

project_dir = 'book_publishing_website'
dockerfile_path = os.path.join(project_dir, 'Dockerfile')

# Instruction 2-8: Define the Dockerfile content
dockerfile_content = """
# Use a base Python image
FROM python:3.9-slim

# Set the working directory
WORKDIR /app

# Copy the requirements file and install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy the rest of the application code
COPY . .

# Expose the port the app runs on (Cloud Run defaults to 8080)
EXPOSE 8080

# Run the application using Gunicorn, binding to 0.0.0.0 and the PORT environment variable
CMD ["gunicorn", "app:app", "--bind", "0.0.0.0:$PORT"]
"""

# Create the Dockerfile
with open(dockerfile_path, 'w') as f:
    f.write(dockerfile_content.strip())

print(f"Created {dockerfile_path}")

# Ensure necessary files for the Docker build exist
# (requirements.txt, app.py, templates, static, schema.sql should already exist from previous steps)
# We will explicitly ensure requirements.txt and app.py exist for robustness.
requirements_content = """
Flask
gunicorn
"""
requirements_file_path = os.path.join(project_dir, 'requirements.txt')
with open(requirements_file_path, 'w') as f:
    f.write(requirements_content.strip())
print(f"Ensured {requirements_file_path} exists.")

app_file_content = """
import sqlite3
import os
from flask import Flask, render_template, request, redirect, url_for, g

app = Flask(__name__)
DATABASE = 'book_publishing_website.db' # Database file inside the container's /app directory

def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        # The database file will be created directly in the WORKDIR (/app)
        db = g._database = sqlite3.connect(DATABASE)
        db.row_factory = sqlite3.Row
    return db

@app.teardown_appcontext
def close_db(error):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

# Function to initialize the database (create table and add some sample data)
# This will run once when the container is built or on the first request if not pre-initialized
def init_db():
    # Check if the database file already exists to avoid re-initializing
    if not os.path.exists(DATABASE):
         with app.app_context():
            db = get_db()
            # Use the schema.sql file from the copied content
            schema_file_path = 'schema.sql' # schema.sql is copied to /app
            with open(schema_file_path, mode='r') as f:
                 db.cursor().executescript(f.read())
            db.commit()
            print("Database initialized.")
    else:
        print("Database already exists.")


# Initialize the database when the application starts
# Note: For Cloud Run with ephemeral storage, this will re-initialize on each new instance.
# A persistent database (like Cloud SQL) is needed for production.
init_db()


@app.route('/')
def index():
    db = get_db()
    query = 'SELECT book_id, title, author FROM books'
    conditions = []
    params = []

    title_search = request.args.get('title')
    author_search = request.args.get('author')
    genre_search = request.args.get('genre')

    if title_search:
        conditions.append('title LIKE ?')
        params.append('%' + title_search + '%')
    if author_search:
        conditions.append('author LIKE ?')
        params.append('%' + author_search + '%')
    if genre_search:
        conditions.append('genre LIKE ?')
        params.append('%' + genre_search + '%')

    if conditions:
        query += ' WHERE ' + ' AND '.join(conditions)

    cursor = db.execute(query, params)
    books = cursor.fetchall()

    return render_template('index.html', books=books, request=request)

@app.route('/book/<int:book_id>')
def book_detail(book_id):
    db = get_db()
    cursor = db.execute('SELECT * FROM books WHERE book_id = ?', (book_id,))
    book = cursor.fetchone()
    if book is None:
        return "Book not found", 404
    return render_template('book_detail.html', book=book)

@app.route('/add_book', methods=('GET', 'POST'))
def add_book():
    if request.method == 'POST':
        title = request.form['title']
        author = request.form['author']
        genre = request.form.get('genre')
        publication_date = request.form.get('publication_date')
        description = request.form.get('description')
        file_path = request.form.get('file_path')
        cover_image_path = request.form.get('cover_image_path')

        db = get_db()
        db.execute(
            'INSERT INTO books (title, author, genre, publication_date, description, file_path, cover_image_path)'
            ' VALUES (?, ?, ?, ?, ?, ?, ?)',
            (title, author, genre, publication_date, description, file_path, cover_image_path)
        )
        db.commit()
        return redirect(url_for('index'))
    return render_template('add_book.html')


# This __main__ block is typically used for local development, not for Gunicorn in production.
# The CMD in the Dockerfile tells Gunicorn how to run the app.
# if __name__ == '__main__':
#     app.run(debug=True)

"""
app_file_path = os.path.join(project_dir, 'app.py')
with open(app_file_path, 'w') as f:
    f.write(app_file_content.strip())
print(f"Ensured {app_file_path} exists and updated database path for container.")

# Ensure schema.sql exists
schema_sql_content = """
DROP TABLE IF EXISTS books;
CREATE TABLE books (
    book_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    genre TEXT,
    publication_date TEXT,
    description TEXT,
    file_path TEXT,
    cover_image_path TEXT
);

INSERT INTO books (title, author, genre, publication_date, description, file_path, cover_image_path) VALUES
('The Great Novel', 'Famous Author', 'Fiction', '2023-01-15', 'A captivating story about life and adventure.', '/books/the_great_novel.pdf', '/static/covers/great_novel_cover.jpg'),
('Mystery of the Old House', 'Intriguing Writer', 'Mystery', '2022-11-20', 'A thrilling mystery set in a spooky old house.', '/books/mystery_old_house.epub', '/static/covers/mystery_house_cover.png'),
('Science of the Future', 'Dr. Innovator', 'Science', '2024-05-10', 'Exploring cutting-edge scientific advancements.', '/books/science_future.pdf', '/static/covers/science_cover.jpg');
"""
schema_file_path = os.path.join(project_dir, 'schema.sql')
with open(schema_file_path, 'w') as f:
    f.write(schema_sql_content)
print(f"Ensured {schema_file_path} exists.")

# Ensure templates directory and content exist
templates_dir = os.path.join(project_dir, 'templates')
os.makedirs(templates_dir, exist_ok=True)
index_html_content = """
<!doctype html>
<html>
<head>
    <title>Published Books</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
</head>
<body>
    <h1>Published Books</h1>

    <form method="GET" action="{{ url_for('index') }}">
        <input type="text" name="title" placeholder="Search by Title" value="{{ request.args.get('title', '') }}">
        <input type="text" name="author" placeholder="Search by Author" value="{{ request.args.get('author', '') }}">
        <input type="text" name="genre" placeholder="Search by Genre" value="{{ request.args.get('genre', '') }}">
        <input type="submit" value="Search">
    </form>

    <p><a href="{{ url_for('add_book') }}">Add New Book</a></p>

    <ul>
    {% for book in books %}
        <li><a href="{{ url_for('book_detail', book_id=book['book_id']) }}">{{ book['title'] }} by {{ book['author'] }}</a></li>
    {% endfor %}
    </ul>
</body>
</html>
"""
index_file_path = os.path.join(templates_dir, 'index.html')
with open(index_file_path, 'w') as f:
    f.write(index_html_content)
print(f"Ensured {index_file_path} exists.")

book_detail_html_content = """
<!doctype html>
<html>
<head>
    <title>{{ book.title }}</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
</head>
<body>
    <a href="{{ url_for('index') }}">Back to Book List</a>
    <h1>{{ book.title }}</h1>
    <p><strong>Author:</strong> {{ book.author }}</p>
    {% if book.genre %}
    <p><strong>Genre:</strong> {{ book.genre }}</p>
    {% endif %}
    {% if book.publication_date %}
    <p><strong>Publication Date:</strong> {{ book.publication_date }}</p>
    {% endif %}
    {% if book.description %}
    <p><strong>Description:</strong></p>
    <p>{{ book.description }}</p>
    {% endif %}
    {% if book.cover_image_path %}
    <p><img src="{{ url_for('static', filename=book.cover_image_path.split('/')[-1]) }}" alt="Cover Image" width="200"></p>
    {% endif %}
    {% if book.file_path %}
    <p><a href="{{ url_for('static', filename=book.file_path.split('/')[-1]) }}">Download Book</a></p>
    {% endif %}
</body>
</html>
"""
book_detail_file_path = os.path.join(templates_dir, 'book_detail.html')
with open(book_detail_file_path, 'w') as f:
    f.write(book_detail_html_content)
print(f"Ensured {book_detail_file_path} exists.")

add_book_html_content = """
<!doctype html>
<html>
<head>
    <title>Add New Book</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
</head>
<body>
    <h1>Add New Book</h1>
    <form method="POST" action="{{ url_for('add_book') }}">
        <label for="title">Title:</label><br>
        <input type="text" id="title" name="title" required><br><br>

        <label for="author">Author:</label><br>
        <input type="text" id="author" name="author" required><br><br>

        <label for="genre">Genre:</label><br>
        <input type="text" id="genre" name="genre"><br><br>

        <label for="publication_date">Publication Date:</label><br>
        <input type="text" id="publication_date" name="publication_date"><br><br>

        <label for="description">Description:</label><br>
        <textarea id="description" name="description"></textarea><br><br>

        <label for="file_path">File Path (e.g., /books/book.pdf):</label><br>
        <input type="text" id="file_path" name="file_path"><br><br>

        <label for="cover_image_path">Cover Image Path (e.g., /static/covers/cover.jpg):</label><br>
        <input type="text" id="cover_image_path" name="cover_image_path"><br><br>

        <input type="submit" value="Add Book">
    </form>
    <p><a href="{{ url_for('index') }}">Back to Book List</a></p>
</body>
</html>
"""
add_book_file_path = os.path.join(templates_dir, 'add_book.html')
with open(add_book_file_path, 'w') as f:
    f.write(add_book_html_content)
print(f"Ensured {add_book_file_path} exists.")


# Ensure static directories and files exist
static_dir = os.path.join(project_dir, 'static')
css_dir = os.path.join(static_dir, 'css')
js_dir = os.path.join(static_dir, 'js') # Optional JS directory

os.makedirs(static_dir, exist_ok=True)
os.makedirs(css_dir, exist_ok=True)
os.makedirs(js_dir, exist_ok=True) # Create JS directory

# Create style.css in the css directory with basic styles
style_css_content = """
body {
    font-family: sans-serif;
    line-height: 1.6;
    margin: 20px;
    background-color: #f4f4f4;
    color: #333;
}

h1, h2 {
    color: #0056b3;
}

a {
    color: #007bff;
    text-decoration: none;
}

a:hover {
    text-decoration: underline;
}

ul {
    list-style: none;
    padding: 0;
}

li {
    background-color: #fff;
    margin-bottom: 10px;
    padding: 15px;
    border-radius: 5px;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
}

form {
    background-color: #fff;
    padding: 20px;
    border-radius: 5px;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
    margin-bottom: 20px;
}

form label {
    display: block;
    margin-bottom: 5px;
    font-weight: bold;
}

form input[type="text"],
form textarea {
    width: 100%;
    padding: 10px;
    margin-bottom: 10px;
    border: 1px solid #ddd;
    border-radius: 4px;
}

form input[type="submit"] {
    background-color: #007bff;
    color: white;
    padding: 10px 15px;
    border: none;
    border-radius: 4px;
    cursor: pointer;
    font-size: 1em;
}

form input[type="submit"]:hover {
    background-color: #0056b3;
}

img {
    max-width: 100%;
    height: auto;
}
"""
style_css_file_path = os.path.join(css_dir, 'style.css')
with open(style_css_file_path, 'w') as f:
    f.write(style_css_content)
print(f"Ensured {style_css_file_path} exists.")


# Create a basic script.js (optional)
script_js_content = """
// Basic JavaScript file
console.log("Script loaded!");
"""
script_js_file_path = os.path.join(js_dir, 'script.js')
with open(script_js_file_path, 'w') as f:
    f.write(script_js_content)
print(f"Ensured {script_js_file_path} exists.")

Created book_publishing_website/Dockerfile
Ensured book_publishing_website/requirements.txt exists.
Ensured book_publishing_website/app.py exists and updated database path for container.
Ensured book_publishing_website/schema.sql exists.
Ensured book_publishing_website/templates/index.html exists.
Ensured book_publishing_website/templates/book_detail.html exists.
Ensured book_publishing_website/templates/add_book.html exists.
Ensured book_publishing_website/static/css/style.css exists.
Ensured book_publishing_website/static/js/script.js exists.


## Set up the Google Cloud environment

### Subtask:
Create a new Google Cloud project, enable necessary APIs, and configure deployment settings.

**Reasoning**:
This step involves actions outside of the notebook environment using the Google Cloud Console and the `gcloud` command-line tool. I will provide instructions on how to perform these manual steps.

**Instructions**:

1.  **Create a new Google Cloud Project:**
    *   Go to the Google Cloud Console ([https://console.cloud.google.com/](https://console.cloud.google.com/)).
    *   Click on the project dropdown at the top of the page.
    *   Click "New Project".
    *   Enter a project name (e.g., `book-publishing-app`) and choose a billing account.
    *   Click "Create".
    *   Make sure the newly created project is selected in the project dropdown.

2.  **Enable necessary APIs:**
    *   In the Google Cloud Console, navigate to the "APIs & Services" > "Library".
    *   Search for "Cloud Run API" and enable it.
    *   Search for "Artifact Registry API" (or "Container Registry API" if you prefer the older service) and enable it. Artifact Registry is the recommended service for storing container images.

3.  **Install and Initialize the `gcloud` CLI:**
    *   If you haven't already, install the Google Cloud CLI ([https://cloud.google.com/sdk/docs/install](https://cloud.google.com/sdk/docs/install)).
    *   Initialize the gcloud environment:

In [ ]:
!gcloud auth configure-docker <your-region>-docker.pkg.dev

/bin/bash: line 1: your-region: No such file or directory


In [ ]:
!gcloud auth configure-docker

`docker` and `docker-credential-gcloud` need to be in the same PATH in order to work correctly together.
gcloud's Docker credential helper can be configured but it will not work until this is corrected.
Adding credentials for all GCR repositories.
After update, the following will be written to your Docker config file located 
at [/root/.docker/config.json]:
 {
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud"
  }
}

Do you want to continue (Y/n)?  

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
the current application. To solve this, set up an application context
with app.app_context(). See the documentation for more information.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google



Command killed by keyboard interrupt

^C


## Configure the database on Google Cloud

### Subtask:
Set up a persistent database solution on Google Cloud for the application's data.

**Reasoning**:
The current SQLite database is not suitable for a production environment on Cloud Run due to its ephemeral nature. Setting up a persistent database like Cloud SQL is necessary to ensure data is saved permanently. This step involves creating a Cloud SQL instance and configuring it.

**Instructions**:

1.  **Create a Cloud SQL Instance:**
    *   Go to the Google Cloud Console ([https://console.cloud.google.com/](https://console.cloud.google.com/)).
    *   Navigate to "Storage" > "Cloud SQL".
    *   Click "Create instance".
    *   Choose a database engine (e.g., **PostgreSQL** or **MySQL** are good choices for web applications).
    *   Choose a region (ideally the same region as your Cloud Run service).
    *   Configure the instance settings (instance ID, password for the default user, machine type, storage, etc.). Start with a small, cost-effective option for testing.
    *   Click "Create". This may take a few minutes.

2.  **Configure Access/Networking:**
    *   Once the instance is created, click on its name to go to its details page.
    *   Go to the "Connections" section.
    *   You'll need to configure how your Cloud Run service will connect to the database. The most secure and recommended way is to use the **Cloud SQL Auth proxy** or configure **VPC Serverless Access**. For simplicity during initial testing, you *could* temporarily allow connections from "All networks" (0.0.0.0/0), but **this is not recommended for production due to security risks**. The Cloud SQL documentation provides detailed guides on secure connection methods.

3.  **Create a Database and User:**
    *   In the Cloud SQL instance details page, go to the "Databases" section and create a new database for your application.
    *   Go to the "Users" section and create a new user with a strong password. Grant this user privileges on the database you just created.

4.  **Initialize the Database Schema:**
    *   You will need to apply your database schema (from your `schema.sql` file) to the new Cloud SQL database. You can do this using a database client (like `psql` for PostgreSQL or `mysql` for MySQL) from your local machine or a Cloud Shell instance that has access to the Cloud SQL instance.
    *   Connect to your Cloud SQL instance using the user and database you created.
    *   Execute the SQL commands from your `schema.sql` file to create the `books` table and insert any initial data.

After completing these steps, your database will be set up on Google Cloud. The next step will be to update your Flask application code to connect to this Cloud SQL instance instead of the local SQLite file.

## Update the application to use the Google Cloud database

### Subtask:
Modify the Flask application to connect to and use the deployed Google Cloud SQL database.

**Reasoning**:
Modify the `app.py` file to replace the SQLite database connection with a connection to the Cloud SQL instance. This requires installing a database connector library (like `psycopg2-binary` for PostgreSQL or `PyMySQL` for MySQL) and updating the `get_db()` function to use the appropriate connection details, likely pulled from environment variables for security.

In [ ]:
import sqlite3 # Keep for potential local testing setup or initial schema loading logic
import os
from flask import Flask, render_template, request, redirect, url_for, g
# Import database connector - install the appropriate one (e.g., psycopg2-binary for PostgreSQL, PyMySQL for MySQL)
# !pip install psycopg2-binary # Uncomment and run this cell if you chose PostgreSQL
# !pip install PyMySQL # Uncomment and run this cell if you chose MySQL

# Assuming you chose PostgreSQL and installed psycopg2-binary
import psycopg2 # Or import pymysql as MySQLdb if you chose MySQL

# Check if app instance already exists
if 'app' not in globals():
    app = Flask(__name__)

# --- Update Database Configuration ---
# For production, use environment variables for sensitive information
# You will set these environment variables in your Cloud Run service configuration
# Example environment variables you might set:
# DB_HOST: The IP address or connection name of your Cloud SQL instance
# DB_USER: The database user you created
# DB_PASSWORD: The password for the database user
# DB_NAME: The name of the database you created

# Replace with your actual environment variable names or configuration method
DB_HOST = os.environ.get('DB_HOST')
DB_USER = os.environ.get('DB_USER')
DB_PASSWORD = os.environ.get('DB_PASSWORD')
DB_NAME = os.environ.get('DB_NAME')

# If you are using the Cloud SQL Auth Proxy with a connection name (recommended for Cloud Run)
# The DB_HOST might be the proxy's address (e.g., '127.0.0.1') and you'd configure the proxy
# to connect to your instance using its connection name (project:region:instance-name).
# Or, if using VPC Serverless Access, DB_HOST would be the private IP.

# --- Updated get_db() function to connect to Cloud SQL ---
def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        # Connect to Cloud SQL using the imported connector
        try:
            # Example for PostgreSQL using psycopg2
            db = g._database = psycopg2.connect(
                host=DB_HOST,
                user=DB_USER,
                password=DB_PASSWORD,
                database=DB_NAME
            )
            # For psycopg2, you typically use cursor.fetchall() which returns tuples.
            # If you want dictionary-like access (like sqlite3.Row), you might need
            # to use a different cursor factory or process the results.
            # For simplicity, we'll adjust the template logic slightly if needed,
            # or you can fetch column names separately.
            print("Connected to Cloud SQL database.")
        except Exception as e:
            print(f"Error connecting to database: {e}")
            # In a real application, you'd handle this error more gracefully
            return None # Or raise the exception

    return db

# --- Keep teardown_appcontext ---
@app.teardown_appcontext
def close_db(error):
    db = getattr(g, '_database', None)
    if db is not None:
        # For database connectors like psycopg2, use close() instead of .close() on the connection object directly
        db.close()
        print("Database connection closed.")


# --- init_db() function (adjust if needed for Cloud SQL) ---
# The init_db function to create the schema and insert initial data is typically
# run *separately* during your deployment process (e.g., in a migration script or
# a one-off container job) after the Cloud SQL instance is ready, not every time
# the application starts in production.
# For now, we'll keep a modified version or note that this is a manual step.
# If you ran the SQL commands manually in Cloud SQL as instructed in the previous step,
# you might not need this function to run within the app container itself.

# If you *do* want to run schema initialization from the app (e.g., for simple setups or testing),
# you would need to adjust it to use the Cloud SQL connection.
# Example (simplified - error handling and migrations are important in production):
def init_db_cloud_sql():
     # This should ideally NOT run every time the app starts in production Cloud Run
     # due to ephemeral storage and potential race conditions if multiple instances start.
     # It's better to run schema initialization as a separate deployment step.
     db = get_db()
     if db:
         try:
             cursor = db.cursor()
             schema_file_path = 'schema.sql' # schema.sql is copied to /app
             with open(schema_file_path, mode='r') as f:
                  cursor.executescript(f.read()) # Note: executescript might not work with all connectors/SQL dialects
             db.commit()
             print("Database initialized (Cloud SQL).")
         except Exception as e:
             print(f"Error initializing database (Cloud SQL): {e}")
         finally:
             cursor.close()
     else:
         print("Could not connect to database for initialization.")

# Decide whether to call init_db_cloud_sql() here.
# For a typical Cloud Run production deployment, you would NOT call this here.
# You would run your schema initialization/migrations as a separate task.
# For demonstration/testing purposes where you might want to quickly set up the schema,
# you *could* call it, but be aware of the limitations mentioned.
# init_db_cloud_sql() # Uncomment ONLY if you understand the implications for ephemeral environments.


# --- Keep Flask Routes (adjust query if necessary based on how data is returned) ---

# Ensure Flask routes are defined only once
if '/' not in [r.rule for r in app.url_map.iter_rules()]:
    @app.route('/')
    def index():
        db = get_db()
        books = [] # Initialize books list
        if db:
            try:
                # Adjust query and fetching based on your database and connector
                # Example for PostgreSQL using psycopg2:
                cursor = db.cursor()
                query = 'SELECT book_id, title, author FROM books'
                conditions = []
                params = []

                title_search = request.args.get('title')
                author_search = request.args.get('author')
                genre_search = request.args.get('genre')

                if title_search:
                    conditions.append('title ILIKE %s') # Use %s for psycopg2 placeholders
                    params.append('%' + title_search + '%')
                if author_search:
                    conditions.append('author ILIKE %s') # Use %s for psycopg2 placeholders
                    params.append('%' + author_search + '%')
                if genre_search:
                    conditions.append('genre ILIKE %s') # Use %s for psycopg2 placeholders
                    params.append('%' + genre_search + '%')

                if conditions:
                    query += ' WHERE ' + ' AND '.join(conditions)

                cursor.execute(query, params)
                books = cursor.fetchall()

                # If using psycopg2's default cursor, books will be a list of tuples.
                # You might need to modify index.html to access elements by index (e.g., book[0])
                # or use a DictCursor if available/desired.
                # If you want dictionary-like access, you can convert tuples to dicts:
                # column_names = [desc[0] for desc in cursor.description]
                # books = [dict(zip(column_names, row)) for row in books]

            except Exception as e:
                print(f"Error fetching books: {e}")
            finally:
                 if 'cursor' in locals() and cursor:
                     cursor.close()


        return render_template('index.html', books=books, request=request)

if '/book/<int:book_id>' not in [r.rule for r in app.url_map.iter_rules()]:
    @app.route('/book/<int:book_id>')
    def book_detail(book_id):
        db = get_db()
        book = None # Initialize book
        if db:
            try:
                # Adjust query and fetching based on your database and connector
                cursor = db.cursor()
                cursor.execute('SELECT * FROM books WHERE book_id = %s', (book_id,)) # Use %s for psycopg2
                book = cursor.fetchone()

                # If using psycopg2's default cursor, book will be a tuple.
                # Convert to dictionary for easier access in template
                if book:
                     column_names = [desc[0] for desc in cursor.description]
                     book = dict(zip(column_names, book))

            except Exception as e:
                print(f"Error fetching book details: {e}")
            finally:
                 if 'cursor' in locals() and cursor:
                     cursor.close()

        if book is None:
            return "Book not found", 404

        return render_template('book_detail.html', book=book)

if '/add_book' not in [r.rule for r in app.url_map.iter_rules()]:
    @app.route('/add_book', methods=('GET', 'POST'))
    def add_book():
        if request.method == 'POST':
            title = request.form['title']
            author = request.form['author']
            genre = request.form.get('genre')
            publication_date = request.form.get('publication_date')
            description = request.form.get('description')
            file_path = request.form.get('file_path')
            cover_image_path = request.form.get('cover_image_path')

            db = get_db()
            if db:
                try:
                    cursor = db.cursor()
                    cursor.execute(
                        'INSERT INTO books (title, author, genre, publication_date, description, file_path, cover_image_path)'
                        ' VALUES (%s, %s, %s, %s, %s, %s, %s)', # Use %s for psycopg2 placeholders
                        (title, author, genre, publication_date, description, file_path, cover_image_path)
                    )
                    db.commit()
                    print("New book added to database.")
                    return redirect(url_for('index'))
                except Exception as e:
                    db.rollback() # Rollback changes if insert fails
                    print(f"Error adding book to database: {e}")
                    # In a real app, show an error message to the user
                    return "Error adding book", 500
                finally:
                    if 'cursor' in locals() and cursor:
                        cursor.close()
            else:
                 return "Database connection error", 500

        return render_template('add_book.html')


# This __main__ block is typically used for local development, not for Gunicorn in production Cloud Run.
# The CMD in the Dockerfile tells Gunicorn how to run the app.
# if __name__ == '__main__':
#     # When running locally for testing with Cloud SQL, you might need to set
#     # the environment variables or configure the Cloud SQL Auth proxy locally.
#     # app.run(debug=True)
#     print("Flask app ready. Note: This script is for defining the app structure. Run with Gunicorn for deployment.")

# Re-create app.py file with the updated content
app_file_path = os.path.join(project_dir, 'app.py')
with open(app_file_path, 'w') as f:
    f.write("""
import sqlite3 # Keep for potential local testing setup or initial schema loading logic
import os
from flask import Flask, render_template, request, redirect, url_for, g
# Import database connector - install the appropriate one (e.g., psycopg2-binary for PostgreSQL or PyMySQL for MySQL)
# !pip install psycopg2-binary # Uncomment and run this cell if you chose PostgreSQL
# !pip install PyMySQL # Uncomment and run this cell if you chose MySQL

# Assuming you chose PostgreSQL and installed psycopg2-binary
import psycopg2 # Or import pymysql as MySQLdb if you chose MySQL

# Check if app instance already exists
if 'app' not in globals():
    app = Flask(__name__)

# --- Update Database Configuration ---
# For production, use environment variables for sensitive information
# You will set these environment variables in your Cloud Run service configuration
# Example environment variables you might set:
# DB_HOST: The IP address or connection name of your Cloud SQL instance
# DB_USER: The database user you created
# DB_PASSWORD: The password for the database user
# DB_NAME: The name of the database you created

# Replace with your actual environment variable names or configuration method
DB_HOST = os.environ.get('DB_HOST')
DB_USER = os.environ.get('DB_USER')
DB_PASSWORD = os.environ.get('DB_PASSWORD')
DB_NAME = os.environ.get('DB_NAME')

# If you are using the Cloud SQL Auth Proxy with a connection name (recommended for Cloud Run)
# The DB_HOST might be the proxy's address (e.g., '127.0.0.1') and you'd configure the proxy
# to connect to your instance using its connection name (project:region:instance-name).
# Or, if using VPC Serverless Access, DB_HOST would be the private IP.

# --- Updated get_db() function to connect to Cloud SQL ---
def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        # Connect to Cloud SQL using the imported connector
        try:
            # Example for PostgreSQL using psycopg2
            db = g._database = psycopg2.connect(
                host=DB_HOST,
                user=DB_USER,
                password=DB_PASSWORD,
                database=DB_NAME
            )
            # For psycopg2, you typically use cursor.fetchall() which returns tuples.
            # If you want dictionary-like access (like sqlite3.Row), you might need
            # to use a different cursor factory or process the results.
            # For simplicity, we'll adjust the template logic slightly if needed,
            # or you can fetch column names separately.
            print("Connected to Cloud SQL database.")
        except Exception as e:
            print(f"Error connecting to database: {e}")
            # In a real application, you'd handle this error more gracefully
            return None # Or raise the exception

    return db

# --- Keep teardown_appcontext ---
@app.teardown_appcontext
def close_db(error):
    db = getattr(g, '_database', None)
    if db is not None:
        # For database connectors like psycopg2, use close() instead of .close() on the connection object directly
        db.close()
        print("Database connection closed.")


# --- init_db() function (adjust if needed for Cloud SQL) ---
# The init_db function to create the schema and insert initial data is typically
# run *separately* during your deployment process (e.g., in a migration script or
# a one-off container job) after the Cloud SQL instance is ready, not every time
# the application starts in production.
# For now, we'll keep a modified version or note that this is a manual step.
# If you ran the SQL commands manually in Cloud SQL as instructed in the previous step,
# you might not need this function to run within the app container itself.

# If you *do* want to run schema initialization from the app (e.g., for simple setups or testing),
# you would need to adjust it to use the Cloud SQL connection.
# Example (simplified - error handling and migrations are important in production):
def init_db_cloud_sql():
     # This should ideally NOT run every time the app starts in production Cloud Run
     # due to ephemeral storage and potential race conditions if multiple instances start.
     # It's better to run schema initialization/migrations as a separate deployment step.
     db = get_db()
     if db:
         try:
             cursor = db.cursor()
             schema_file_path = 'schema.sql' # schema.sql is copied to /app
             with open(schema_file_path, mode='r') as f:
                  cursor.executescript(f.read()) # Note: executescript might not work with all connectors/SQL dialects
             db.commit()
             print("Database initialized (Cloud SQL).")
         except Exception as e:
             print(f"Error initializing database (Cloud SQL): {e}")
         finally:
             if 'cursor' in locals() and cursor:
                 cursor.close()
     else:
         print("Could not connect to database for initialization.")

# Decide whether to call init_db_cloud_sql() here.
# For a typical Cloud Run production deployment, you would NOT call this here.
# You would run your schema initialization/migrations as a separate task.
# For demonstration/testing purposes where you might want to quickly set up the schema,
# you *could* call it, but be aware of the limitations mentioned.
# init_db_cloud_sql() # Uncomment ONLY if you understand the implications for ephemeral environments.


# --- Keep Flask Routes (adjust query if necessary based on how data is returned) ---

# Ensure Flask routes are defined only once
if '/' not in [r.rule for r in app.url_map.iter_rules()]:
    @app.route('/')
    def index():
        db = get_db()
        books = [] # Initialize books list
        if db:
            try:
                # Adjust query and fetching based on your database and connector
                # Example for PostgreSQL using psycopg2:
                cursor = db.cursor()
                query = 'SELECT book_id, title, author FROM books'
                conditions = []
                params = []

                title_search = request.args.get('title')
                author_search = request.args.get('author')
                genre_search = request.args.get('genre')

                if title_search:
                    conditions.append('title ILIKE %s') # Use %s for psycopg2 placeholders
                    params.append('%' + title_search + '%')
                if author_search:
                    conditions.append('author ILIKE %s') # Use %s for psycopg2 placeholders
                    params.append('%' + author_search + '%')
                if genre_search:
                    conditions.append('genre ILIKE %s') # Use %s for psycopg2 placeholders
                    params.append('%' + genre_search + '%')

                if conditions:
                    query += ' WHERE ' + ' AND '.join(conditions)

                cursor.execute(query, params)
                books = cursor.fetchall()

                # If using psycopg2's default cursor, books will be a list of tuples.
                # You might need to modify index.html to access elements by index (e.g., book[0])
                # or use a DictCursor if available/desired.
                # If you want dictionary-like access, you can convert tuples to dicts:
                # column_names = [desc[0] for desc in cursor.description]
                # books = [dict(zip(column_names, row)) for row in books]

            except Exception as e:
                print(f"Error fetching books: {e}")
            finally:
                 if 'cursor' in locals() and cursor:
                     cursor.close()


        return render_template('index.html', books=books, request=request)

if '/book/<int:book_id>' not in [r.rule for r in app.url_map.iter_rules()]:
    @app.route('/book/<int:book_id>')
    def book_detail(book_id):
        db = get_db()
        book = None # Initialize book
        if db:
            try:
                # Adjust query and fetching based on your database and connector
                cursor = db.cursor()
                cursor.execute('SELECT * FROM books WHERE book_id = %s', (book_id,)) # Use %s for psycopg2
                book = cursor.fetchone()

                # If using psycopg2's default cursor, book will be a tuple.
                # Convert to dictionary for easier access in template
                if book:
                     column_names = [desc[0] for desc in cursor.description]
                     book = dict(zip(column_names, book))

            except Exception as e:
                print(f"Error fetching book details: {e}")
            finally:
                 if 'cursor' in locals() and cursor:
                     cursor.close()


        if book is None:
            return "Book not found", 404

        return render_template('book_detail.html', book=book)

if '/add_book' not in [r.rule for r in app.url_map.iter_rules()]:
    @app.route('/add_book', methods=('GET', 'POST'))
    def add_book():
        if request.method == 'POST':
            title = request.form['title']
            author = request.form['author']
            genre = request.form.get('genre')
            publication_date = request.form.get('publication_date')
            description = request.form.get('description')
            file_path = request.form.get('file_path')
            cover_image_path = request.form.get('cover_image_path')

            db = get_db()
            if db:
                try:
                    cursor = db.cursor()
                    cursor.execute(
                        'INSERT INTO books (title, author, genre, publication_date, description, file_path, cover_image_path)'
                        ' VALUES (%s, %s, %s, %s, %s, %s, %s)', # Use %s for psycopg2 placeholders
                        (title, author, genre, publication_date, description, file_path, cover_image_path)
                    )
                    db.commit()
                    print("New book added to database.")
                    return redirect(url_for('index'))
                except Exception as e:
                    db.rollback() # Rollback changes if insert fails
                    print(f"Error adding book to database: {e}")
                    # In a real app, show an error message to the user
                    return "Error adding book", 500
                finally:
                    if 'cursor' in locals() and cursor:
                        cursor.close()
            else:
                 return "Database connection error", 500


# This __main__ block is typically used for local development, not for Gunicorn in production Cloud Run.
# The CMD in the Dockerfile tells Gunicorn how to run the app.
# if __name__ == '__main__':
#     # When running locally for testing with Cloud SQL, you might need to set
#     # the environment variables or configure the Cloud SQL Auth proxy locally.
#     # app.run(debug=True)
#     print("Flask app ready. Note: This script is for defining the app structure. Run with Gunicorn for deployment.")

""")
print(f"Updated {app_file_path} to connect to Cloud SQL.")
print("Remember to install the database connector library (e.g., psycopg2-binary or PyMySQL) and set the environment variables (DB_HOST, DB_USER, DB_PASSWORD, DB_NAME) in your Cloud Run service configuration.")

Updated book_publishing_website/app.py to connect to Cloud SQL.
Remember to install the database connector library (e.g., psycopg2-binary or PyMySQL) and set the environment variables (DB_HOST, DB_USER, DB_PASSWORD, DB_NAME) in your Cloud Run service configuration.
